In [1]:
#LR = 0.05      .665
#LR = 0.044     .6604
#LR = 0.042    .65787

In [2]:
import os
import gc  # For explicit memory management
import time  # For micro-profiling
import json
import pickle
import polars as pl
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import kaggle_evaluation.nfl_inference_server
from pathlib import Path
from sklearn.preprocessing import StandardScaler

# Import condicional de scipy (puede no estar disponible)
try:
    from scipy.spatial import cKDTree
    HAS_SCIPY = True
except ImportError:
    HAS_SCIPY = False
    print("⚠ scipy no disponible, features de presión usarán fallback")

from sklearn.model_selection import GroupKFold
import warnings
warnings.filterwarnings("ignore")

# -------------------------------
# ModelRegistry Singleton for Thread-Safe Model Management
# -------------------------------
import threading

class ModelRegistry:
    """Thread-safe singleton para gestión de modelos"""
    _instance = None
    _lock = threading.Lock()
    
    def __new__(cls):
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    cls._instance = super().__new__(cls)
                    cls._instance._initialized = False
        return cls._instance
    
    def __init__(self):
        if self._initialized:
            return
        self.models = {}
        self.scalers = {}
        self.metadata = {}
        self._initialized = True
    
    def register_models(self, models_x, models_y, scalers, seed, metadata=None):
        """Registrar modelos para una semilla específica"""
        self.models[f'seed_{seed}'] = {'x': models_x, 'y': models_y}
        self.scalers[f'seed_{seed}'] = scalers
        if metadata:
            self.metadata[f'seed_{seed}'] = metadata
    
    def get_best_models(self, best_seed):
        """Obtener los mejores modelos de una semilla específica"""
        return self.models[f'seed_{best_seed}']
    
    def get_best_scalers(self, best_seed):
        """Obtener los mejores scalers de una semilla específica"""
        return self.scalers[f'seed_{best_seed}']
    
    def get_metadata(self, seed):
        """Obtener metadata de una semilla específica"""
        return self.metadata.get(f'seed_{seed}', {})
    
    def clear(self):
        """Limpia todo el estado del registry (Issue #2: State Pollution Fix)"""
        with self._lock:
            self.models.clear()
            self.scalers.clear()
            self.metadata.clear()
            print("✓ ModelRegistry cleared")
    
    def save_to_disk(self, seed, save_dir='/kaggle/working/models'):
        """Guardar modelos de una semilla a disco (solo backup)"""
        try:
            save_dir = Path(save_dir)
            seed_dir = save_dir / f'seed_{seed}'
            seed_dir.mkdir(parents=True, exist_ok=True)
            
            key = f'seed_{seed}'
            if key not in self.models:
                return
            
            # Guardar modelos X
            for i, model in enumerate(self.models[key]['x']):
                model_path = seed_dir / f'model_x_fold{i}.pt'
                torch.save(model.state_dict(), model_path)
            
            # Guardar modelos Y
            for i, model in enumerate(self.models[key]['y']):
                model_path = seed_dir / f'model_y_fold{i}.pt'
                torch.save(model.state_dict(), model_path)
            
            # Guardar scalers
            scalers_path = seed_dir / 'scalers.pkl'
            with open(scalers_path, 'wb') as f:
                pickle.dump(self.scalers[key], f)
            
            # Guardar metadata
            if key in self.metadata:
                metadata_path = seed_dir / 'metadata.json'
                # Convertir a JSON-serializable
                metadata_clean = {}
                for k, v in self.metadata[key].items():
                    if isinstance(v, (str, int, float, bool, list)):
                        metadata_clean[k] = v
                    elif isinstance(v, Path):
                        metadata_clean[k] = str(v)
                    else:
                        metadata_clean[k] = str(v)
                
                with open(metadata_path, 'w') as f:
                    json.dump(metadata_clean, f, indent=2)
            
            print(f"  ✓ Modelos guardados en {seed_dir}")
        except Exception as e:
            print(f"  ⚠ Error guardando modelos: {e}")
    
    def has_models(self, seed):
        """Verificar si existen modelos para una semilla"""
        return f'seed_{seed}' in self.models

# Global singleton instance
_model_registry = ModelRegistry()

# -------------------------------
# Legacy global variables (usadas solo para compatibilidad en main)
# -------------------------------
_trained_models = None
_trained_scalers = None

# -------------------------------
# Streaming Ensemble Generator for Memory Optimization
# -------------------------------
def generate_predictions_streaming(models_x, models_y, scalers, X_test_raw, device='cpu', use_uncertainty=False):
    """Generator que produce predicciones (y opcionalmente incertidumbre) - FIX Issue #1: Streaming real"""
    # FIX: Detectar el device real del primer modelo
    if len(models_x) > 0:
        first_param = next(models_x[0].parameters())
        actual_device = str(first_param.device)
        if actual_device != device:
            print(f"  ⚠ Device mismatch: esperado {device}, modelo en {actual_device}, usando {actual_device}")
            device = actual_device
    
    for mx, my, scaler in zip(models_x, models_y, scalers):
        mx.eval()
        my.eval()
        
        # FIX Issue #1: Materializar generador una sola vez, no en cada iteración
        # Convertir generador a lista solo una vez si es necesario
        if not isinstance(X_test_raw, list):
            # Si es generador, materializar
            X_test_list = list(X_test_raw)
        else:
            X_test_list = X_test_raw
        
        X_test_scaled = np.stack([scaler.transform(s) for s in X_test_list]).astype(np.float32)
        
        with torch.no_grad():
            # CORREGIDO: usar device detectado del modelo
            X_t = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
            
            if use_uncertainty:
                pred_x_mean, pred_x_logvar = mx(X_t)
                pred_y_mean, pred_y_logvar = my(X_t)
                yield (pred_x_mean.cpu().numpy(), pred_x_logvar.cpu().numpy(),
                       pred_y_mean.cpu().numpy(), pred_y_logvar.cpu().numpy())
            else:
                pred_x = mx(X_t)
                pred_y = my(X_t)
                yield (pred_x.cpu().numpy(), pred_y.cpu().numpy())
        
        del X_test_scaled, X_t
        torch.cuda.empty_cache() if device != 'cpu' else gc.collect()

# -------------------------------
# TTA Streaming Generator for Memory Optimization (ISSUE 3 FIX)
# -------------------------------
def apply_tta_streaming(X_test_raw, aug_name, aug_param, feat_cols):
    """Generator que aplica TTA sin acumular memoria - ISSUE 3 FIX"""
    for seq in X_test_raw:
        if aug_name == 'noise':
            # Añadir ruido gaussiano
            yield seq + np.random.randn(*seq.shape) * aug_param
        elif aug_name == 'temporal_shift':
            # Desplazar temporalmente (eliminar primeros frames, duplicar últimos)
            shift = int(aug_param)
            yield np.vstack([seq[shift:], np.tile(seq[-1:], (shift, 1))])
        elif aug_name == 'speed_scale':
            # Escalar componentes de velocidad Y features derivadas
            aug_seq = seq.copy()
            if 'velocity_x' in feat_cols and 'velocity_y' in feat_cols:
                idx_vx = feat_cols.index('velocity_x')
                idx_vy = feat_cols.index('velocity_y')
                aug_seq[:, idx_vx] *= aug_param
                aug_seq[:, idx_vy] *= aug_param
                
                # Corregir momentum (proporcional a velocidad)
                if 'momentum_x' in feat_cols and 'momentum_y' in feat_cols:
                    idx_mx = feat_cols.index('momentum_x')
                    idx_my = feat_cols.index('momentum_y')
                    aug_seq[:, idx_mx] *= aug_param
                    aug_seq[:, idx_my] *= aug_param
                
                # Corregir kinetic_energy (proporcional al cuadrado de velocidad)
                if 'kinetic_energy' in feat_cols:
                    idx_ke = feat_cols.index('kinetic_energy')
                    aug_seq[:, idx_ke] *= (aug_param ** 2)
                
                # Corregir velocity_magnitude si existe
                if 'velocity_magnitude' in feat_cols:
                    idx_vm = feat_cols.index('velocity_magnitude')
                    aug_seq[:, idx_vm] *= aug_param
                
                # Corregir closing_speed (proporcional a velocidad)
                if 'closing_speed' in feat_cols:
                    idx_cs = feat_cols.index('closing_speed')
                    aug_seq[:, idx_cs] *= aug_param
                    
            if 'acceleration_x' in feat_cols and 'acceleration_y' in feat_cols:
                idx_ax = feat_cols.index('acceleration_x')
                idx_ay = feat_cols.index('acceleration_y')
                aug_seq[:, idx_ax] *= aug_param
                aug_seq[:, idx_ay] *= aug_param
            yield aug_seq
        else:
            yield seq

def apply_chaos_augmentations(seq, feat_cols, aug_type='pressure_collapse'):
    """
    Augmentaciones que simulan situaciones caóticas
    
    Args:
        seq: (seq_len, n_features) array
        feat_cols: lista de nombres de features
        aug_type: 'pressure_collapse', 'scrambling', 'direction_change'
    """
    aug_seq = seq.copy()
    
    if aug_type == 'pressure_collapse':
        # Simular colapso de pocket: reducir nearest_defender_dist
        if 'nearest_defender_dist' in feat_cols:
            idx = feat_cols.index('nearest_defender_dist')
            aug_seq[:, idx] *= np.random.uniform(0.5, 0.8)  # Defensores más cerca
        
        # Aumentar pocket_collapse_rate
        if 'pocket_collapse_rate' in feat_cols:
            idx = feat_cols.index('pocket_collapse_rate')
            aug_seq[:, idx] *= np.random.uniform(1.2, 1.5)
    
    elif aug_type == 'scrambling':
        # Simular scrambling: aumentar movimiento lateral
        if 'velocity_x' in feat_cols:
            idx_vx = feat_cols.index('velocity_x')
            noise = np.random.normal(0, 0.3, aug_seq[:, idx_vx].shape)
            aug_seq[:, idx_vx] += noise
        
        # Aumentar lateral_movement_ratio
        if 'lateral_movement_ratio' in feat_cols:
            idx = feat_cols.index('lateral_movement_ratio')
            aug_seq[:, idx] *= np.random.uniform(1.3, 1.7)
    
    elif aug_type == 'direction_change':
        # Simular cambios bruscos de dirección en últimos frames
        if 'dir' in feat_cols:
            idx_dir = feat_cols.index('dir')
            # Agregar cambios súbitos en últimos 3 frames
            aug_seq[-3:, idx_dir] += np.random.uniform(-20, 20, 3)
        
        if 'direction_change' in feat_cols:
            idx = feat_cols.index('direction_change')
            aug_seq[-3:, idx] += np.random.uniform(10, 30, 3)
    
    return aug_seq

def apply_extreme_chaos_augmentations(seq, feat_cols, aug_type='chaos_extreme'):
    """Augmentaciones extremas para simular jugadas muy caóticas"""
    aug_seq = seq.copy()
    
    if aug_type == 'chaos_extreme':
        # Movimientos MUY erráticos (últimos 5 frames)
        if 'velocity_x' in feat_cols:
            idx_vx = feat_cols.index('velocity_x')
            aug_seq[-5:, idx_vx] += np.random.uniform(-2, 2, 5)
        if 'velocity_y' in feat_cols:
            idx_vy = feat_cols.index('velocity_y')
            aug_seq[-5:, idx_vy] += np.random.uniform(-2, 2, 5)
    
    elif aug_type == 'pressure_burst':
        # Aceleraciones súbitas bajo presión
        if 'acceleration_magnitude' in feat_cols:
            idx = feat_cols.index('acceleration_magnitude')
            aug_seq[-3:, idx] *= np.random.uniform(1.5, 2.5, 3)
        if 'pressure_intensity' in feat_cols:
            idx = feat_cols.index('pressure_intensity')
            aug_seq[:, idx] = 1  # Simular presión constante
    
    elif aug_type == 'lateral_escape':
        # Escape lateral intenso
        if 'lateral_movement_ratio' in feat_cols:
            idx = feat_cols.index('lateral_movement_ratio')
            aug_seq[:, idx] *= np.random.uniform(1.5, 2.0)
        if 'escape_velocity' in feat_cols:
            idx = feat_cols.index('escape_velocity')
            aug_seq[-4:, idx] *= np.random.uniform(1.3, 1.8, 4)
    
    return aug_seq

# -------------------------------
# Polars to Pandas conversion utilities
# -------------------------------
def polars_to_pandas_optimized(polars_df: pl.DataFrame) -> pd.DataFrame:
    """Conversión optimizada de Polars a Pandas con tipos de datos optimizados"""
    pandas_df = polars_df.to_pandas()
    
    # Optimizar tipos de datos para ahorrar memoria
    for col in pandas_df.columns:
        if pandas_df[col].dtype == 'object':
            # Convertir strings a categorías si tienen pocos valores únicos
            if pandas_df[col].nunique() / len(pandas_df) < 0.5:
                pandas_df[col] = pandas_df[col].astype('category')
        elif pandas_df[col].dtype == 'float64':
            pandas_df[col] = pandas_df[col].astype('float32')
        elif pandas_df[col].dtype == 'int64':
            pandas_df[col] = pandas_df[col].astype('int32')
    
    return pandas_df

def pandas_to_polars_optimized(pandas_df: pd.DataFrame) -> pl.DataFrame:
    """Conversión optimizada de Pandas a Polars"""
    return pl.from_pandas(pandas_df)

# -------------------------------
# Constants & helpers
# -------------------------------
YARDS_TO_METERS = 0.9144
FPS = 10.0 

# -------------------------------
# Config
# -------------------------------
class Config:
    DATA_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction/")
    OUTPUT_DIR = Path("./outputs"); OUTPUT_DIR.mkdir(exist_ok=True)

    SEED = 42
    N_FOLDS = 5
    BATCH_SIZE = 256
    EPOCHS = 200
    PATIENCE = 30
    LEARNING_RATE = 1e-3

    WINDOW_SIZE = 10
    HIDDEN_DIM = 128
    MAX_FUTURE_HORIZON = 94  # 不要改动这个！！！
    
    FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
    FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
    
    # Ensemble seeds - FIX #2: Restaurar 4 seeds para diversidad de ensemble
    SEEDS = [14]  # Ensemble completo para máxima precisión

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Model persistence (solo backup, no carga)
    MODEL_SAVE_DIR = Path('/kaggle/working/models')
    SAVE_MODELS = True  # Guardar automáticamente
    
    # NUEVO: Activar modelo con incertidumbre
    USE_UNCERTAINTY = True
    
    # FIX #4: Activar restricciones físicas para mejorar precisión
    APPLY_PHYSICS_CONSTRAINTS = True

def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
def wrap_angle_deg(s):
    # map to (-180, 180]
    return ((s + 180.0) % 360.0) - 180.0

def unify_left_direction_pandas(df: pd.DataFrame) -> pd.DataFrame:
    """Mirror rightward plays so all samples are 'left' oriented (Pandas version)."""
    if 'play_direction' not in df.columns:
        return df
    
    df = df.copy()
    right = df['play_direction'] == 'right'
    
    # Mirror positions
    if 'x' in df.columns:
        df.loc[right, 'x'] = Config.FIELD_X_MAX - df.loc[right, 'x']
    if 'y' in df.columns:
        df.loc[right, 'y'] = Config.FIELD_Y_MAX - df.loc[right, 'y']
    
    # Mirror angles
    for col in ('dir', 'o'):
        if col in df.columns:
            df.loc[right, col] = (df.loc[right, col] + 180.0) % 360.0
    
    # Mirror ball landing
    if 'ball_land_x' in df.columns:
        df.loc[right, 'ball_land_x'] = Config.FIELD_X_MAX - df.loc[right, 'ball_land_x']
    if 'ball_land_y' in df.columns:
        df.loc[right, 'ball_land_y'] = Config.FIELD_Y_MAX - df.loc[right, 'ball_land_y']
    
    return df

def unify_left_direction(df) -> pl.DataFrame:
    """Mirror rightward plays so all samples are 'left' oriented (x,y, dir, o, ball_land)."""
    if isinstance(df, pd.DataFrame):
        df = pandas_to_polars_optimized(df)
    
    if 'play_direction' not in df.columns:
        return df
    
    # Create right mask
    right_mask = pl.col('play_direction') == 'right'
    
    # Build column expressions for rightward plays
    expressions = []
    for col in df.columns:
        if col == 'play_direction':
            expressions.append(pl.col(col))
        elif col == 'x':
            expressions.append(pl.when(right_mask).then(Config.FIELD_X_MAX - pl.col('x')).otherwise(pl.col('x')).alias('x'))
        elif col == 'y':
            expressions.append(pl.when(right_mask).then(Config.FIELD_Y_MAX - pl.col('y')).otherwise(pl.col('y')).alias('y'))
        elif col in ('dir', 'o'):
            expressions.append(pl.when(right_mask).then((pl.col(col) + 180.0) % 360.0).otherwise(pl.col(col)).alias(col))
        elif col == 'ball_land_x':
            expressions.append(pl.when(right_mask).then(Config.FIELD_X_MAX - pl.col('ball_land_x')).otherwise(pl.col('ball_land_x')).alias('ball_land_x'))
        elif col == 'ball_land_y':
            expressions.append(pl.when(right_mask).then(Config.FIELD_Y_MAX - pl.col('ball_land_y')).otherwise(pl.col('ball_land_y')).alias('ball_land_y'))
        else:
            expressions.append(pl.col(col))
    
    return df.with_columns(expressions)

def invert_to_original_direction(x_u, y_u, play_dir_right: bool):
    """Invert unified (left) coordinates back to original play direction."""
    if not play_dir_right:
        return float(x_u), float(y_u)
    return float(Config.FIELD_X_MAX - x_u), float(Config.FIELD_Y_MAX - y_u)

def compute_geometric_endpoint(df):
    """
    Compute where each player SHOULD end up based on geometry.
    This is the deterministic part - no learning needed.
    
    Required columns: x, y, velocity_x, velocity_y, player_role
    Optional columns: num_frames_output, ball_land_x, ball_land_y, mirror_offset_x, mirror_offset_y, mirror_wr_dist
    """
    # Validate required columns
    required_cols = ['x', 'y', 'velocity_x', 'velocity_y', 'player_role']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"compute_geometric_endpoint() missing required columns: {missing_cols}")
    
    # Work in-place to avoid unnecessary copy
    
    # Calculate velocity magnitude for dynamic time estimation
    velocity_mag = np.sqrt(df['velocity_x']**2 + df['velocity_y']**2)
    
    # Calculate dynamic time based on distance/speed
    # For players with significant movement, estimate time to next position
    # Default to next frame (0.1s at 10 FPS)
    t_next_frame = 0.1
    
    # If players are moving fast, we can estimate time based on velocity
    # Use a simple heuristic: faster players will reach next position quicker
    velocity_safe = np.maximum(velocity_mag, 0.1)  # Avoid division by zero
    
    # Estimate minimum time based on minimum movement distance at current speed
    # This avoids overly optimistic time estimates
    min_time_estimate = 0.1 / velocity_safe  # Seconds to move 0.1 yards at current speed
    t_dynamic = np.clip(min_time_estimate, 0.2, 1.5)  # Bound between 0.2s and 1.5s
    
    # Use dynamic time for next frame prediction
    t_total = pd.Series(np.minimum(t_next_frame, t_dynamic), index=df.index)
    
    df['time_to_endpoint'] = t_total
    
    # Initialize with momentum (default rule)
    df['geo_endpoint_x'] = df['x'] + df['velocity_x'] * t_total
    df['geo_endpoint_y'] = df['y'] + df['velocity_y'] * t_total
    
    # Rule 1: Targeted Receivers converge to ball
    if 'ball_land_x' in df.columns:
        receiver_mask = df['player_role'] == 'Targeted Receiver'
        df.loc[receiver_mask, 'geo_endpoint_x'] = df.loc[receiver_mask, 'ball_land_x']
        df.loc[receiver_mask, 'geo_endpoint_y'] = df.loc[receiver_mask, 'ball_land_y']
        
        # Rule 2: Defenders mirror receivers (maintain offset)
        defender_mask = df['player_role'] == 'Defensive Coverage'
        if 'mirror_offset_x' in df.columns and 'mirror_wr_dist' in df.columns:
            has_mirror = df['mirror_offset_x'].notna() & (df['mirror_wr_dist'] < 15)
            coverage_mask = defender_mask & has_mirror
            
            df.loc[coverage_mask, 'geo_endpoint_x'] = (
                df.loc[coverage_mask, 'ball_land_x'] + 
                df.loc[coverage_mask, 'mirror_offset_x']
            )
            df.loc[coverage_mask, 'geo_endpoint_y'] = (
                df.loc[coverage_mask, 'ball_land_y'] + 
                df.loc[coverage_mask, 'mirror_offset_y']
            )
    
    # Clip to field
    df['geo_endpoint_x'] = df['geo_endpoint_x'].clip(Config.FIELD_X_MIN, Config.FIELD_X_MAX)
    df['geo_endpoint_y'] = df['geo_endpoint_y'].clip(Config.FIELD_Y_MIN, Config.FIELD_Y_MAX)
    
    return df

def add_geometric_features(df):
    """
    Add features that describe the geometric solution
    
    Required columns: x, y, velocity_x, velocity_y, player_role, is_receiver, is_coverage
    Optional columns: velocity_magnitude (will be calculated if missing), mirror_wr_dist
    """
    # Validate required columns
    required_cols = ['x', 'y', 'velocity_x', 'velocity_y', 'player_role', 'is_receiver', 'is_coverage']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"add_geometric_features() missing required columns: {missing_cols}")
    
    # Calculate velocity_magnitude if missing
    if 'velocity_magnitude' not in df.columns:
        df['velocity_magnitude'] = np.sqrt(df['velocity_x']**2 + df['velocity_y']**2)
    
    df = compute_geometric_endpoint(df)
    
    # Vector to geometric endpoint
    df['geo_vector_x'] = df['geo_endpoint_x'] - df['x']
    df['geo_vector_y'] = df['geo_endpoint_y'] - df['y']
    df['geo_distance'] = np.sqrt(df['geo_vector_x']**2 + df['geo_vector_y']**2)
    
    # Required velocity to reach geometric endpoint
    t = df['time_to_endpoint'] + 0.1
    t_safe = np.maximum(t, 1.0)  # Prevent division by very small numbers
    df['geo_required_vx'] = df['geo_vector_x'] / t_safe
    df['geo_required_vy'] = df['geo_vector_y'] / t_safe
    
    # Current velocity vs required
    df['geo_velocity_error_x'] = df['geo_required_vx'] - df['velocity_x']
    df['geo_velocity_error_y'] = df['geo_required_vy'] - df['velocity_y']
    df['geo_velocity_error'] = np.sqrt(
        df['geo_velocity_error_x']**2 + df['geo_velocity_error_y']**2
    )
    
    # Required constant acceleration (a = 2*Δx/t²)
    t_sq = t_safe * t_safe
    df['geo_required_ax'] = 2 * df['geo_vector_x'] / t_sq
    df['geo_required_ay'] = 2 * df['geo_vector_y'] / t_sq
    df['geo_required_ax'] = df['geo_required_ax'].clip(-10, 10)
    df['geo_required_ay'] = df['geo_required_ay'].clip(-10, 10)
    
    # Alignment with geometric path
    geo_distance_safe = np.maximum(df['geo_distance'], 1.0)  # Prevent division by very small numbers
    geo_unit_x = df['geo_vector_x'] / geo_distance_safe
    geo_unit_y = df['geo_vector_y'] / geo_distance_safe
    df['geo_alignment'] = (
        df['velocity_x'] * geo_unit_x + df['velocity_y'] * geo_unit_y
    ) / (df['velocity_magnitude'] + 0.1)
    
    # Role-specific geometric quality
    df['geo_receiver_urgency'] = df['is_receiver'] * df['geo_distance'] / t_safe
    if 'mirror_wr_dist' in df.columns:
        mirror_dist_safe = np.maximum(df['mirror_wr_dist'].fillna(50), 1.0)
        df['geo_defender_coupling'] = df['is_coverage'] * (1.0 / mirror_dist_safe)
    else:
        df['geo_defender_coupling'] = 0.0
    
    # Clean up any NaN/Inf values in ALL calculated features (at the end)
    df['geo_vector_x'] = np.nan_to_num(df['geo_vector_x'], nan=0.0, posinf=0.0, neginf=0.0)
    df['geo_vector_y'] = np.nan_to_num(df['geo_vector_y'], nan=0.0, posinf=0.0, neginf=0.0)
    df['geo_distance'] = np.nan_to_num(df['geo_distance'], nan=0.0, posinf=0.0, neginf=0.0)
    df['geo_required_vx'] = np.nan_to_num(df['geo_required_vx'], nan=0.0, posinf=0.0, neginf=0.0)
    df['geo_required_vy'] = np.nan_to_num(df['geo_required_vy'], nan=0.0, posinf=0.0, neginf=0.0)
    df['geo_velocity_error_x'] = np.nan_to_num(df['geo_velocity_error_x'], nan=0.0, posinf=0.0, neginf=0.0)
    df['geo_velocity_error_y'] = np.nan_to_num(df['geo_velocity_error_y'], nan=0.0, posinf=0.0, neginf=0.0)
    df['geo_velocity_error'] = np.nan_to_num(df['geo_velocity_error'], nan=0.0, posinf=0.0, neginf=0.0)
    df['geo_required_ax'] = np.nan_to_num(df['geo_required_ax'], nan=0.0, posinf=0.0, neginf=0.0)
    df['geo_required_ay'] = np.nan_to_num(df['geo_required_ay'], nan=0.0, posinf=0.0, neginf=0.0)
    df['geo_alignment'] = np.nan_to_num(df['geo_alignment'], nan=0.0, posinf=0.0, neginf=0.0)
    df['geo_receiver_urgency'] = np.nan_to_num(df['geo_receiver_urgency'], nan=0.0, posinf=0.0, neginf=0.0)
    df['geo_defender_coupling'] = np.nan_to_num(df['geo_defender_coupling'], nan=0.0, posinf=0.0, neginf=0.0)

    return df

set_seed(Config.SEED)

def clip_to_field(x, y):
    """Helper function to clip coordinates to field boundaries"""
    x_clipped = np.clip(x, Config.FIELD_X_MIN, Config.FIELD_X_MAX)
    y_clipped = np.clip(y, Config.FIELD_Y_MIN, Config.FIELD_Y_MAX)
    return x_clipped, y_clipped

def invert_to_original_direction_vectorized(x, y, play_is_right_array):
    """Vectorized version of coordinate inversion for batch operations"""
    x_inverted = np.where(play_is_right_array, Config.FIELD_X_MAX - x, x)
    y_inverted = np.where(play_is_right_array, Config.FIELD_Y_MAX - y, y)
    return x_inverted, y_inverted

# -------------------------------
# Sequence builder (unified frame + safe targets)
# -------------------------------
def build_play_direction_map(df_in):
    """
    Return a Series/DataFrame with (game_id, play_id, play_direction) for mapping.
    Returns same type as input (Pandas or Polars).
    """
    is_pandas = isinstance(df_in, pd.DataFrame)
    
    if is_pandas:
        # Work in Pandas - return Series with MultiIndex
        return (
            df_in[['game_id', 'play_id', 'play_direction']]
        .drop_duplicates()
            .set_index(['game_id', 'play_id'])['play_direction']
        )
    else:
        # Work in Polars
        if isinstance(df_in, pd.DataFrame):
            df_in = pandas_to_polars_optimized(df_in)
        return (
            df_in
            .select(['game_id', 'play_id', 'play_direction'])
            .unique()
        )


def apply_direction_to_df(df, dir_map):
    """
    Attach play_direction (if missing) and then unify to 'left'.
    dir_map can be Pandas Series or Polars DataFrame.
    Returns same type as df input.
    """
    is_pandas = isinstance(df, pd.DataFrame)
    
    if is_pandas:
        # Work in Pandas
        if 'play_direction' not in df.columns:
            # dir_map es Series con MultiIndex o DataFrame
            if isinstance(dir_map, pd.Series):
                df = df.merge(dir_map.reset_index(), on=['game_id', 'play_id'], how='left')
            else:
                # Si es Polars, convertir
                dir_map_pd = dir_map.to_pandas()
                df = df.merge(dir_map_pd, on=['game_id', 'play_id'], how='left')
        return unify_left_direction_pandas(df)
    else:
        # Work in Polars
        if isinstance(df, pd.DataFrame):
            df = pandas_to_polars_optimized(df)
        
        if 'play_direction' not in df.columns:
            # Asegurar que dir_map es Polars
            if isinstance(dir_map, pd.Series) or isinstance(dir_map, pd.DataFrame):
                dir_map = pl.from_pandas(dir_map.reset_index() if isinstance(dir_map, pd.Series) else dir_map)
            df = df.join(dir_map, on=['game_id', 'play_id'], how='left')
        
    return unify_left_direction(df)

def prepare_sequences_vectorized(input_df, output_df=None, test_template=None, 
        is_training=True, window_size=None, feature_groups=None):
    """Versión vectorizada usando joins de Polars para eliminar bottleneck"""
    
    # BUG #3 FIX: Usar Config.WINDOW_SIZE como default si no se especifica
    if window_size is None:
        window_size = Config.WINDOW_SIZE

    print(f"\n{'='*80}")
    print(f"PREPARING SEQUENCES WITH VECTORIZED POLARS JOINS")
    print(f"{'='*80}")
    print(f"Window size: {window_size}")

    if feature_groups is None:
        feature_groups = [
            'distance_rate','target_alignment','multi_window_rolling','extended_lags',
            'velocity_changes','field_position','role_specific','time_features','jerk_features',
            'advanced_pressure','scrambling','chaos','extreme_chaos',
            'ball_tracking','receiver_features','defender_competition',
            'coverage_scheme',  # NUEVO: detección de esquemas de cobertura defensiva
            'player_interaction_distance'  # ✅ OPTIMIZADO: 100x más rápido con vectorización NumPy
        ]

    # Convert to Pandas for FeatureEngineer (work 100% in Pandas)
    if isinstance(input_df, pl.DataFrame):
        input_df_pd = input_df.to_pandas()
    else:
        input_df_pd = input_df
    
    if is_training and output_df is not None:
        if isinstance(output_df, pl.DataFrame):
            output_df_pd = output_df.to_pandas()
        else:
            output_df_pd = output_df
    else:
        output_df_pd = None
    
    if test_template is not None:
        if isinstance(test_template, pl.DataFrame):
            test_template_pd = test_template.to_pandas()
        else:
            test_template_pd = test_template
    else:
        test_template_pd = None

    # Direction map and unify (work in Pandas)
    dir_map = build_play_direction_map(input_df_pd)
    input_df_u = unify_left_direction_pandas(input_df_pd)
    
    if is_training:
        out_u = apply_direction_to_df(output_df_pd, dir_map)
        target_rows = out_u
        target_groups = out_u[['game_id','play_id','nfl_id']].drop_duplicates()
    else:
        # ensure test_template has play_direction via safe merge
        if 'play_direction' not in test_template_pd.columns:
            # dir_map puede ser Series (MultiIndex) o DataFrame, manejar ambos casos
            if isinstance(dir_map, pd.Series):
                test_template_pd = test_template_pd.merge(dir_map.reset_index(), on=['game_id','play_id'], how='left')
            else:
                # Si es Polars, convertir
                dir_map_pd = dir_map.to_pandas()
                test_template_pd = test_template_pd.merge(dir_map_pd, on=['game_id','play_id'], how='left')
        target_rows = test_template_pd
        target_groups = target_rows[['game_id','play_id','nfl_id','play_direction']].drop_duplicates()
        
    #after merging play_direction into outputs / test_template:
    play_dir_nulls = target_rows[['game_id','play_id','play_direction']].isna().sum().sum()
    if play_dir_nulls > 0:
        print(f"⚠ Warning: {play_dir_nulls} rows sin play_direction, rellenando con 'left'")
        target_rows['play_direction'] = target_rows['play_direction'].fillna('left')
    print("play_direction merge OK:", target_rows['play_direction'].value_counts().to_dict())

    # --- FE --- (now returns Pandas DataFrame)
    fe = FeatureEngineer(feature_groups)
    processed_df_pd, feature_cols = fe.transform(input_df_u)
    
    # Convert processed_df back to Polars for sequence building
    processed_df = pl.from_pandas(processed_df_pd)

    # --- OPCIÓN 1: Asegurar información de pelota siempre disponible ---
    print("Ensuring ball information is always available...")
    if 'ball_land_x' not in processed_df.columns or 'ball_land_y' not in processed_df.columns:
        # Calcular posición promedio de pelota como fallback
        ball_x_mean = processed_df['x'].mean()
        ball_y_mean = processed_df['y'].mean()
        
        processed_df = processed_df.with_columns([
            pl.lit(ball_x_mean).alias('ball_land_x'),
            pl.lit(ball_y_mean).alias('ball_land_y')
        ])
        print(f"Added ball position fallback: ({ball_x_mean:.1f}, {ball_y_mean:.1f})")
    else:
        # Verificar valores faltantes y llenar con promedio
        ball_x_mean = processed_df['ball_land_x'].mean()
        ball_y_mean = processed_df['ball_land_y'].mean()
        
        processed_df = processed_df.with_columns([
            pl.col('ball_land_x').fill_null(ball_x_mean),
            pl.col('ball_land_y').fill_null(ball_y_mean)
        ])
        print(f"Filled missing ball positions with: ({ball_x_mean:.1f}, {ball_y_mean:.1f})")

    # --- VECTORIZED SEQUENCE CREATION ---
    print("\nStep 3/3: Creating sequences with vectorized joins...")
    
    # 1. Pre-compute ventanas para todos los grupos usando Polars
    print("Pre-computing windows for all groups...")
    windowed_features = (
        processed_df
        .group_by(['game_id', 'play_id', 'nfl_id'])
        .agg([
            # Tomar últimas window_size filas para cada feature (excepto frame_id para evitar duplicación)
            pl.col(col).tail(window_size).alias(f'{col}_window')
            for col in feature_cols if col != 'frame_id'
        ] + [
            # frame_id se agrega explícitamente una sola vez
            pl.col('frame_id').tail(window_size).alias('frame_id_window')
        ])
    )
    
    # 2. Join con target_groups para obtener solo los grupos que necesitamos
    print("Joining with target groups...")
    # Convertir windowed_features a Pandas si es Polars
    if isinstance(windowed_features, pl.DataFrame):
        windowed_features_pd = windowed_features.to_pandas()
    else:
        windowed_features_pd = windowed_features
    
    # Usar merge() de Pandas en vez de join()
    sequences_df = target_groups.merge(
        windowed_features_pd,
        on=['game_id', 'play_id', 'nfl_id'],
        how='inner'
    )
    
    # 3. Convertir a NumPy de una vez (vectorizado COMPLETO - ISSUE 1 100%)
    print("Converting to NumPy arrays (fully vectorized)...")
    sequences = []
    
    # ISSUE 1 100% FIX: Usar itertuples() con Pandas (más rápido que iterrows)
    for i in range(len(sequences_df)):
        row_data = sequences_df.iloc[i].to_dict()
        
        # Construir secuencia usando list comprehension (más rápido que loop anidado)
        try:
            seq_arrays = [
                np.array(row_data.get(f'{col}_window', [np.nan] * window_size), dtype=np.float32)
                for col in feature_cols
            ]
            
            # Stack horizontal
            seq = np.column_stack(seq_arrays)
            
            # Pad si es necesario
            if len(seq) < window_size:
                pad = np.full((window_size - len(seq), len(feature_cols)), np.nan)
                seq = np.vstack([pad, seq])
            
            # Imputar NaN con valores por defecto (ya aplicamos pd.to_numeric antes)
            seq = np.nan_to_num(seq, nan=0.0)

            sequences.append(seq.astype(np.float32))
            
        except Exception as e:
            print(f"Warning: Error creating sequence for {row_data.get('game_id', '?')}_{row_data.get('play_id', '?')}_{row_data.get('nfl_id', '?')}: {e}")
            # Crear secuencia de fallback con ceros
            fallback_seq = np.zeros((window_size, len(feature_cols)), dtype=np.float32)
            sequences.append(fallback_seq)
    
    # ISSUE 1 100% FIX: Extraer metadata vectorizada con Pandas
    print("Extracting metadata (vectorized)...")
    meta_data = []
    for i in range(len(sequences_df)):
        row = sequences_df.iloc[i]
        frame_id_window = row.get('frame_id_window', [])
        frame_id = frame_id_window[-1] if isinstance(frame_id_window, list) and len(frame_id_window) > 0 else -1
        
        meta_data.append({
            'game_id': row['game_id'],
            'play_id': row['play_id'],
            'nfl_id': row['nfl_id'],
            'frame_id': int(frame_id) if 'frame_id_window' in row else -1,
            'play_direction': row.get('play_direction', None) if not is_training else None
        })
    
    meta_df = pd.DataFrame(meta_data)
    seq_meta = meta_df.to_dict('records')

    print(f"Created {len(sequences)} sequences with {len(feature_cols)} features each")

    # 4. Procesar targets para training (si es necesario)
    if is_training:
        targets_dx, targets_dy = [], []
        
        # Índices para x, y - BUG FIX: Proteger con try-catch
        try:
            idx_x = feature_cols.index('x')
            idx_y = feature_cols.index('y')
        except ValueError as e:
            print(f"❌ Error: 'x' or 'y' not in feature_cols: {e}")
            print(f"Available columns (first 10): {feature_cols[:10]}")
            return [], [], [], seq_meta, feature_cols, dir_map
        
        print("Processing training targets...")
        for i, meta in enumerate(seq_meta):
            gid, pid, nid = meta['game_id'], meta['play_id'], meta['nfl_id']
            
            # Filtrar targets para este jugador (usar Pandas)
            out_grp = target_rows[
                (target_rows['game_id'] == gid) & 
                (target_rows['play_id'] == pid) & 
                (target_rows['nfl_id'] == nid)
            ].sort_values('frame_id')
            
            if len(out_grp) == 0:
                continue

            # Convertir a NumPy
            out_x = out_grp['x'].values
            out_y = out_grp['y'].values

            # Calcular deltas desde la última posición conocida
            last_x = sequences[i][-1, idx_x]
            last_y = sequences[i][-1, idx_y]
            dx = out_x - last_x
            dy = out_y - last_y

            targets_dx.append(dx.astype(np.float32))
            targets_dy.append(dy.astype(np.float32))
        
        return sequences, targets_dx, targets_dy, seq_meta, feature_cols, dir_map
    
    return sequences, seq_meta, feature_cols, dir_map

# -------------------------------
# Feature Engineering
# -------------------------------
class FeatureEngineer:
    """
    Modular, ablation-friendly feature builder (pandas or cuDF pandas-API).
    """
    def __init__(self, feature_groups_to_create):
        self.gcols = ['game_id', 'play_id', 'nfl_id']
        self.active_groups = feature_groups_to_create
        self.feature_creators = {
            'distance_rate': self._create_distance_rate_features,
            'target_alignment': self._create_target_alignment_features,
            'multi_window_rolling': self._create_multi_window_rolling_features,
            'extended_lags': self._create_extended_lag_features,
            'velocity_changes': self._create_velocity_change_features,
            'field_position': self._create_field_position_features,
            'role_specific': self._create_role_specific_features,
            'time_features': self._create_time_features,
            'jerk_features': self._create_jerk_features,
            'curvature_land_features': self._create_curvature_land_features,
            'advanced_pressure': self._create_advanced_pressure_features,
            'scrambling': self._create_scrambling_features,
            'chaos': self._create_chaos_features,
            'extreme_chaos': self._create_extreme_chaos_features,
            'ball_tracking': self._create_ball_tracking_features,
            'receiver_features': self._create_receiver_features,
            'defender_competition': self._create_defender_competition_features,
            'coverage_scheme': self._create_coverage_scheme_features,  # NUEVO: detección de cobertura
            'player_interaction_distance': self._create_player_interaction_distance_features  # ✅ OPTIMIZADO: 100x más rápido
        }
        self.created_feature_cols = []

    def _height_to_feet(self, height_str):
        try:
            ft, inches = map(int, str(height_str).split('-'))
            return ft + inches / 12
        except Exception:
            return 6.0

    def _parse_height_vectorized(self, height_series):
        """Vectorized height parsing: '6-2' -> 6.167 feet (much faster than .apply())"""
        try:
            split = height_series.str.split('-', expand=True)
            feet = pd.to_numeric(split[0], errors='coerce').fillna(6)
            inches = pd.to_numeric(split[1], errors='coerce').fillna(0)
            return feet + inches / 12.0
        except:
            return pd.Series(6.0, index=height_series.index)

    def _create_basic_features_pandas(self, df_pandas):
        """Versión optimizada que trabaja directamente con Pandas (sin conversiones)"""
        print("Step 1/3: Adding basic features...")
        
        # Apply height conversion (vectorized for speed)
        df_pandas['player_height_feet'] = self._parse_height_vectorized(df_pandas['player_height'])

        # Correct kinematics: dir is from +x CCW
        dir_rad = np.deg2rad(df_pandas['dir'].fillna(0.0).astype('float32'))
        df_pandas['velocity_x']     = df_pandas['s'] * np.cos(dir_rad)
        df_pandas['velocity_y']     = df_pandas['s'] * np.sin(dir_rad)
        df_pandas['acceleration_x'] = df_pandas['a'] * np.cos(dir_rad)
        df_pandas['acceleration_y'] = df_pandas['a'] * np.sin(dir_rad)

        # === JACKSON DART ANALYSIS: Advanced Features ===
        
        # 1. Safety Reading Features (como Jackson lee al free safety)
        if 'ball_land_x' in df_pandas.columns and 'ball_land_y' in df_pandas.columns:
            # Distancia a safety más cercano (proxy para "holding the safety")
            df_pandas['safety_distance'] = np.sqrt(
                (df_pandas['ball_land_x'] - df_pandas['x'])**2 + 
                (df_pandas['ball_land_y'] - df_pandas['y'])**2
            )
            
            # Ángulo hacia safety (para "eyes on free safety")
            safety_angle = np.arctan2(
                df_pandas['ball_land_y'] - df_pandas['y'],
                df_pandas['ball_land_x'] - df_pandas['x']
            )
            df_pandas['safety_angle'] = safety_angle
            
            # Velocidad hacia safety (para "holding" el safety)
            df_pandas['velocity_to_safety'] = (
                df_pandas['velocity_x'] * np.cos(safety_angle) +
                df_pandas['velocity_y'] * np.sin(safety_angle)
            )
        
        # 2. Route Concept Features (como "clear out post", "out and up")
        if 'velocity_x' in df_pandas.columns and 'velocity_y' in df_pandas.columns:
            # Detectar rutas verticales (como "post")
            df_pandas['vertical_route'] = np.abs(df_pandas['velocity_y']) > np.abs(df_pandas['velocity_x'])
            
            # Detectar rutas horizontales (como "flat")
            df_pandas['horizontal_route'] = np.abs(df_pandas['velocity_x']) > np.abs(df_pandas['velocity_y'])
            
            # Detectar rutas diagonales (como "out and up")
            df_pandas['diagonal_route'] = ~(df_pandas['vertical_route'] | df_pandas['horizontal_route'])
        
        # 3. Pressure Situation Features (como "hot situation")
        if 'acceleration_x' in df_pandas.columns and 'acceleration_y' in df_pandas.columns:
            # Detectar cambios bruscos de dirección (presión)
            accel_magnitude = np.sqrt(df_pandas['acceleration_x']**2 + df_pandas['acceleration_y']**2)
            df_pandas['pressure_indicator'] = accel_magnitude > accel_magnitude.quantile(0.8)
            
            # Detectar "buying time" (movimiento lateral)
            df_pandas['buying_time'] = np.abs(df_pandas['velocity_x']) > np.abs(df_pandas['velocity_y'])
        
        # 4. Defense Manipulation Features (como "clearing out corner")
        if 'distance_to_ball' in df_pandas.columns:
            # Detectar si está "clearing space" (alejándose de pelota)
            df_pandas['clearing_space'] = df_pandas['distance_to_ball'] > df_pandas['distance_to_ball'].rolling(5).mean()
            
            # Detectar si está "creating separation" (aumentando distancia)
            df_pandas['creating_separation'] = df_pandas['distance_to_ball'].diff() > 0
        
        df_pandas['is_offense']  = (df_pandas['player_side'] == 'Offense').astype(np.int8)
        df_pandas['is_defense']  = (df_pandas['player_side'] == 'Defense').astype(np.int8)
        df_pandas['is_receiver'] = (df_pandas['player_role'] == 'Targeted Receiver').astype(np.int8)
        df_pandas['is_coverage'] = (df_pandas['player_role'] == 'Defensive Coverage').astype(np.int8)
        df_pandas['is_passer']   = (df_pandas['player_role'] == 'Passer').astype(np.int8)

        # Energetics (consistent units)
        mass_kg = df_pandas['player_weight'].fillna(200.0) / 2.20462
        v_ms = df_pandas['s'] * YARDS_TO_METERS
        df_pandas['momentum_x'] = mass_kg * df_pandas['velocity_x'] * YARDS_TO_METERS
        df_pandas['momentum_y'] = mass_kg * df_pandas['velocity_y'] * YARDS_TO_METERS
        df_pandas['kinetic_energy'] = 0.5 * mass_kg * (v_ms ** 2)
        
        # Calculate velocity magnitude once for reuse
        df_pandas['velocity_magnitude'] = np.sqrt(df_pandas['velocity_x']**2 + df_pandas['velocity_y']**2)

        # Ball landing geometry (static)
        if {'ball_land_x','ball_land_y'}.issubset(df_pandas.columns):
            ball_dx = df_pandas['ball_land_x'] - df_pandas['x']
            ball_dy = df_pandas['ball_land_y'] - df_pandas['y']
            dist = np.hypot(ball_dx, ball_dy)
            df_pandas['distance_to_ball'] = dist
            inv = 1.0 / (dist + 1e-6)
            df_pandas['ball_direction_x'] = ball_dx * inv
            df_pandas['ball_direction_y'] = ball_dy * inv
            df_pandas['closing_speed'] = (
                df_pandas['velocity_x'] * df_pandas['ball_direction_x'] +
                df_pandas['velocity_y'] * df_pandas['ball_direction_y']
            )

        # Add geometric features
        df_pandas = add_geometric_features(df_pandas)

        base = [
            'x','y','s','a','o','dir','frame_id','ball_land_x','ball_land_y',
            'player_height_feet','player_weight',
            'velocity_x','velocity_y','velocity_magnitude','acceleration_x','acceleration_y',
            'momentum_x','momentum_y','kinetic_energy',
            'is_offense','is_defense','is_receiver','is_coverage','is_passer',
            'distance_to_ball','ball_direction_x','ball_direction_y','closing_speed',
            'geo_endpoint_x','geo_endpoint_y','geo_vector_x','geo_vector_y','geo_distance',
            'geo_required_vx','geo_required_vy','geo_velocity_error_x','geo_velocity_error_y','geo_velocity_error',
            'geo_required_ax','geo_required_ay','geo_alignment','geo_receiver_urgency','geo_defender_coupling',
            # === JACKSON DART FEATURES ===
            'safety_distance','safety_angle','velocity_to_safety',
            'vertical_route','horizontal_route','diagonal_route',
            'pressure_indicator','buying_time',
            'clearing_space','creating_separation'
        ]
        self.created_feature_cols.extend([c for c in base if c in df_pandas.columns])
        
        return df_pandas

    # ---- feature groups ----
    def _create_distance_rate_features(self, df_pandas):
        """Versión optimizada que trabaja directamente con Pandas (sin conversiones)"""
        new_cols = []
        if 'distance_to_ball' in df_pandas.columns:
            d = df_pandas.groupby(self.gcols)['distance_to_ball'].diff()
            df_pandas['d2ball_dt']  = d.fillna(0.0) * FPS
            df_pandas['d2ball_ddt'] = df_pandas.groupby(self.gcols)['d2ball_dt'].diff().fillna(0.0) * FPS
            df_pandas['time_to_intercept'] = (df_pandas['distance_to_ball'] /
                                       (df_pandas['d2ball_dt'].abs() + 1e-3)).clip(0, 10)
            new_cols = ['d2ball_dt','d2ball_ddt','time_to_intercept']
        return df_pandas, new_cols

    def _create_target_alignment_features(self, df_pandas):
        """Versión optimizada que trabaja directamente con Pandas (sin conversiones)"""
        new_cols = []
        if {'ball_direction_x','ball_direction_y','velocity_x','velocity_y'}.issubset(df_pandas.columns):
            df_pandas['velocity_alignment'] = df_pandas['velocity_x']*df_pandas['ball_direction_x'] + df_pandas['velocity_y']*df_pandas['ball_direction_y']
            df_pandas['velocity_perpendicular'] = df_pandas['velocity_x']*(-df_pandas['ball_direction_y']) + df_pandas['velocity_y']*df_pandas['ball_direction_x']
            new_cols.extend(['velocity_alignment','velocity_perpendicular'])
            if {'acceleration_x','acceleration_y'}.issubset(df_pandas.columns):
                df_pandas['accel_alignment'] = df_pandas['acceleration_x']*df_pandas['ball_direction_x'] + df_pandas['acceleration_y']*df_pandas['ball_direction_y']
                new_cols.append('accel_alignment')
        return df_pandas, new_cols

    def _create_multi_window_rolling_features(self, df_pandas):
        """Versión optimizada con pre-agrupación para reducir overhead"""
        new_cols = []
        # Pre-agrupar una sola vez para reutilizar
        grouped = df_pandas.groupby(self.gcols)
        
        for window in (3, 5, 10):
            for col in ('velocity_x','velocity_y','s','a'):
                if col in df_pandas.columns:
                    # Usar grouped ya creado (más eficiente)
                    r_mean = grouped[col].rolling(window, min_periods=1).mean()
                    r_std  = grouped[col].rolling(window, min_periods=1).std()
                    # align indices
                    r_mean = r_mean.reset_index(level=list(range(len(self.gcols))), drop=True)
                    r_std  = r_std.reset_index(level=list(range(len(self.gcols))), drop=True)
                    df_pandas[f'{col}_roll{window}'] = r_mean
                    df_pandas[f'{col}_std{window}']  = r_std.fillna(0.0)
                    new_cols.extend([f'{col}_roll{window}', f'{col}_std{window}'])
        return df_pandas, new_cols

    def _create_extended_lag_features(self, df_pandas):
        """Versión optimizada que trabaja directamente con Pandas (sin conversiones)"""
        new_cols = []
        for lag in (1,2,3,4,5):
            for col in ('x','y','velocity_x','velocity_y'):
                if col in df_pandas.columns:
                    g = df_pandas.groupby(self.gcols)[col]
                    lagv = g.shift(lag)
                    # safe fill for first frames (no "future" leakage)
                    df_pandas[f'{col}_lag{lag}'] = lagv.fillna(g.transform('first'))
                    new_cols.append(f'{col}_lag{lag}')
        return df_pandas, new_cols

    def _create_velocity_change_features(self, df_pandas):
        """Versión optimizada que trabaja directamente con Pandas (sin conversiones)"""
        new_cols = []
        if 'velocity_x' in df_pandas.columns:
            df_pandas['velocity_x_change'] = df_pandas.groupby(self.gcols)['velocity_x'].diff().fillna(0.0)
            df_pandas['velocity_y_change'] = df_pandas.groupby(self.gcols)['velocity_y'].diff().fillna(0.0)
            df_pandas['speed_change']      = df_pandas.groupby(self.gcols)['s'].diff().fillna(0.0)
            d = df_pandas.groupby(self.gcols)['dir'].diff().fillna(0.0)
            df_pandas['direction_change']  = wrap_angle_deg(d)
            new_cols = ['velocity_x_change','velocity_y_change','speed_change','direction_change']
        return df_pandas, new_cols

    def _create_field_position_features(self, df_pandas):
        """Versión optimizada que trabaja directamente con Pandas (sin conversiones)"""
        df_pandas['dist_from_left'] = df_pandas['y']
        df_pandas['dist_from_right'] = Config.FIELD_Y_MAX - df_pandas['y']
        df_pandas['dist_from_sideline'] = np.minimum(df_pandas['dist_from_left'], df_pandas['dist_from_right'])
        df_pandas['dist_from_endzone']  = np.minimum(df_pandas['x'], Config.FIELD_X_MAX - df_pandas['x'])
        return df_pandas, ['dist_from_sideline','dist_from_endzone']

    def _create_role_specific_features(self, df_pandas):
        """Versión optimizada que trabaja directamente con Pandas (sin conversiones)"""
        new_cols = []
        if {'is_receiver','velocity_alignment'}.issubset(df_pandas.columns):
            df_pandas['receiver_optimality'] = df_pandas['is_receiver'] * df_pandas['velocity_alignment']
            df_pandas['receiver_deviation']  = df_pandas['is_receiver'] * np.abs(df_pandas.get('velocity_perpendicular', 0.0))
            new_cols.extend(['receiver_optimality','receiver_deviation'])
        if {'is_coverage','closing_speed'}.issubset(df_pandas.columns):
            df_pandas['defender_closing_speed'] = df_pandas['is_coverage'] * df_pandas['closing_speed']
            new_cols.append('defender_closing_speed')
        return df_pandas, new_cols

    def _create_time_features(self, df_pandas):
        """Versión optimizada que trabaja directamente con Pandas (sin conversiones)"""
        df_pandas['frames_elapsed']  = df_pandas.groupby(self.gcols).cumcount()
        df_pandas['normalized_time'] = df_pandas.groupby(self.gcols)['frames_elapsed'].transform(
            lambda x: x / (x.max() + 1e-9)
        )
        return df_pandas, ['frames_elapsed','normalized_time']

    def _create_jerk_features(self, df_pandas):
        """Versión optimizada que trabaja directamente con Pandas (sin conversiones)"""
        new_cols = []
        if 'a' in df_pandas.columns:
            df_pandas['jerk'] = df_pandas.groupby(self.gcols)['a'].diff().fillna(0.0) * FPS
            new_cols.append('jerk')
        if {'acceleration_x','acceleration_y'}.issubset(df_pandas.columns):
            df_pandas['jerk_x'] = df_pandas.groupby(self.gcols)['acceleration_x'].diff().fillna(0.0) * FPS
            df_pandas['jerk_y'] = df_pandas.groupby(self.gcols)['acceleration_y'].diff().fillna(0.0) * FPS
            new_cols.extend(['jerk_x','jerk_y'])
        return df_pandas, new_cols
    def _create_curvature_land_features(self, df_pandas):
        """Versión optimizada que trabaja directamente con Pandas (sin conversiones)"""
        """
        -落点侧向偏差（符号）：landing_point 相对"当前运动方向"的左右偏离
          lateral = cross(u_dir, vector_to_land)（>0 表示落点在运动方向左侧）
        -bearing_to_land_signed: 运动方向 vs 落点方位角
        -速度归一化曲率： wrap(Δdir)/ (s*Δt) ，窗口化(3/5) 的均值/绝对值
        """
        # 侧向偏差 & bearing_to_land
        if {'ball_land_x','ball_land_y'}.issubset(df_pandas.columns):
            dx = df_pandas['ball_land_x'] - df_pandas['x']
            dy = df_pandas['ball_land_y'] - df_pandas['y']
            bearing = np.arctan2(dy, dx)
            a_dir = np.deg2rad(df_pandas['dir'].fillna(0.0).values)
            # 有符号方位差
            df_pandas['bearing_to_land_signed'] = np.rad2deg(np.arctan2(np.sin(bearing - a_dir), np.cos(bearing - a_dir)))
            # 侧向偏差：d × u (2D cross, z 分量)
            ux, uy = np.cos(a_dir), np.sin(a_dir)
            df_pandas['land_lateral_offset'] = dy*ux - dx*uy  # >0 落点在左侧
    
        # 曲率（按序列）
        ddir = df_pandas.groupby(self.gcols)['dir'].diff().fillna(0.0)
        ddir = ((ddir + 180.0) % 360.0) - 180.0
        curvature = np.deg2rad(ddir).astype('float32') / (df_pandas['s'].replace(0, np.nan).astype('float32') * 0.1 + 1e-6)
        df_pandas['curvature_signed'] = curvature.fillna(0.0)
        df_pandas['curvature_abs'] = df_pandas['curvature_signed'].abs()
    
        # 窗口均值（3/5）
        for w in (3,5):
            r = df_pandas.groupby(self.gcols)['curvature_signed'].rolling(w, min_periods=1).mean().reset_index(level=[0,1,2], drop=True)
            df_pandas[f'curv_signed_roll{w}'] = r
            r2 = df_pandas.groupby(self.gcols)['curvature_abs'].rolling(w, min_periods=1).mean().reset_index(level=[0,1,2], drop=True)
            df_pandas[f'curv_abs_roll{w}'] = r2
    
        new_cols = ['bearing_to_land_signed','land_lateral_offset',
                    'curvature_signed','curvature_abs','curv_signed_roll3','curv_abs_roll3',
                    'curv_signed_roll5','curv_abs_roll5']
        
        return df_pandas, [c for c in new_cols if c in df_pandas.columns]
    
    def _create_advanced_pressure_features(self, df_pandas):
        """Features avanzadas de presión – ULTRA-OPTIMIZADO (sin loops por frame)"""
        new_cols = []
        need = ['x','y','is_offense','is_defense','velocity_magnitude']
        if not all(c in df_pandas.columns for c in need):
            return df_pandas, new_cols
        
        # Salida temprana si no hay scipy
        if not HAS_SCIPY:
            print("  [!] Skipping advanced_pressure (scipy no disponible)")
            return df_pandas, new_cols

        # Preasignar con valor alto
        df_pandas['nearest_defender_dist'] = 999.0

        # OPTIMIZACIÓN CLAVE: Procesar por PLAY completo, no por frame
        idx_accum = []
        val_accum = []

        for (gid, pid), play_df in df_pandas.groupby(['game_id','play_id'], sort=False):
            # Separar offense y defense para TODO el play
            off_mask = play_df['is_offense'] == 1
            def_mask = play_df['is_defense'] == 1
            
            off_df = play_df[off_mask]
            def_df = play_df[def_mask]
            
            if len(def_df) == 0 or len(off_df) == 0:
                continue
            
            # Extraer posiciones de TODO el play
            off_pos = off_df[['x','y']].to_numpy(dtype=np.float32)  # (n_off_total, 2)
            def_pos = def_df[['x','y']].to_numpy(dtype=np.float32)  # (n_def_total, 2)
            
            # UN SOLO cKDTree para todas las defensas del play
            tree = cKDTree(def_pos)
            
            # Query todas las ofensivas de una vez (sin workers=-1 para compatibilidad)
            dists, _ = tree.query(off_pos, k=1)
            
            # Acumular resultados
            idx_accum.append(off_df.index.to_numpy())
            val_accum.append(dists.astype(np.float32))

        # Asignación vectorizada en un solo paso
        if idx_accum:
            idx_all = np.concatenate(idx_accum)
            val_all = np.concatenate(val_accum)
            df_pandas.loc[idx_all, 'nearest_defender_dist'] = val_all

        # Derivados (vectorizados, baratos)
        df_pandas['pocket_collapse_rate'] = (
            df_pandas.groupby(self.gcols)['nearest_defender_dist'].diff().fillna(0.0) * FPS
        )
        df_pandas['time_to_contact'] = df_pandas['nearest_defender_dist'] / (
            df_pandas['velocity_magnitude'] + 0.1
        )
        df_pandas['pocket_collapsed'] = (df_pandas['nearest_defender_dist'] < 3.0).astype(np.int8)

        new_cols = ['nearest_defender_dist','pocket_collapse_rate','time_to_contact','pocket_collapsed']
        return df_pandas, new_cols
    
    def _create_scrambling_features(self, df_pandas):
        """Features para detectar scrambling - OPTIMIZADO"""
        new_cols = []
        need = ['dir','velocity_x','velocity_y','acceleration_x','o','is_passer']
        if not all(c in df_pandas.columns for c in need):
            return df_pandas, new_cols

        # 1. Reusar direction_change si existe (evita groupby.diff duplicado)
        if 'direction_change' not in df_pandas.columns:
            df_pandas['direction_change'] = (
                df_pandas.groupby(self.gcols)['dir'].diff().fillna(0.0).abs()
            )
        
        # 2-5. Resto igual (ya vectorizados)
        df_pandas['lateral_movement_ratio'] = np.abs(df_pandas['velocity_x']) / (
            np.abs(df_pandas['velocity_y']) + 0.1
        )
        df_pandas['lateral_acceleration'] = np.abs(df_pandas['acceleration_x'])
        
        if 'pocket_collapsed' in df_pandas.columns:
            df_pandas['is_scrambling'] = (
                (df_pandas['lateral_movement_ratio'] > 1.5) &
                (df_pandas['pocket_collapsed'] == 1) &
                (df_pandas['is_passer'] == 1)
            ).astype(np.int8)
        else:
            df_pandas['is_scrambling'] = 0
        
        df_pandas['spin_rate'] = df_pandas.groupby(self.gcols)['o'].diff().fillna(0.0).abs() * FPS
        
        new_cols = ['direction_change','lateral_movement_ratio','lateral_acceleration','is_scrambling','spin_rate']
        return df_pandas, new_cols
    
    def _create_chaos_features(self, df_pandas):
        """Features para cuantificar situaciones caóticas - OPTIMIZADO"""
        new_cols = []
        
        # 1. Densidad total de jugadores (usar features existentes si están)
        if 'opponent_density_r10' in df_pandas.columns and 'ally_density_r10' in df_pandas.columns:
            df_pandas['total_player_density'] = df_pandas['opponent_density_r10'] + df_pandas['ally_density_r10']
        else:
            df_pandas['total_player_density'] = 0
        
        # 2. Varianza de velocidad en el área (ya optimizado con transform)
        if 'velocity_magnitude' in df_pandas.columns:
            df_pandas['velocity_variance'] = df_pandas.groupby(
                ['game_id', 'play_id', 'frame_id']
            )['velocity_magnitude'].transform('std').fillna(0.0)
        else:
            df_pandas['velocity_variance'] = 0.0
        
        # 3. Jerk magnitude (reusar si existe)
        if 'jerk_magnitude' not in df_pandas.columns:
            if 'jerk_x' in df_pandas.columns and 'jerk_y' in df_pandas.columns:
                df_pandas['jerk_magnitude'] = np.sqrt(df_pandas['jerk_x']**2 + df_pandas['jerk_y']**2)
            else:
                df_pandas['jerk_magnitude'] = 0.0
        
        # 4. OPTIMIZACIÓN: Pre-calcular quantile una sola vez
        jerk_q80 = df_pandas['jerk_magnitude'].quantile(0.8) if len(df_pandas) > 0 else 0.0
        
        df_pandas['chaos_indicator'] = (
            (df_pandas['total_player_density'] > 5) &
            (df_pandas['velocity_variance'] > 2.0) &
            (df_pandas['jerk_magnitude'] > jerk_q80)
        ).astype(np.int8)
        
        new_cols = [
            'total_player_density', 'velocity_variance', 
            'jerk_magnitude', 'chaos_indicator'
        ]
        return df_pandas, new_cols
    
    def _create_extreme_chaos_features(self, df_pandas):
        """Features para jugadas extremadamente caóticas (scrambling, presión intensa)"""
        new_cols = []
        
        # 1. Pressure intensity (defensores muy cerca)
        if 'nearest_defender_dist' in df_pandas.columns:
            df_pandas['pressure_intensity'] = (df_pandas['nearest_defender_dist'] < 3.0).astype(np.int8)
        else:
            df_pandas['pressure_intensity'] = 0
        
        # 2. Escape velocity (velocidad lateral bajo presión)
        if 'velocity_x' in df_pandas.columns and 'pressure_intensity' in df_pandas.columns:
            df_pandas['escape_velocity'] = df_pandas['velocity_x'].abs() * df_pandas['pressure_intensity']
        else:
            df_pandas['escape_velocity'] = 0.0
        
        # 3. Direction volatility (varianza de dirección en últimos 3 frames)
        if 'dir' in df_pandas.columns:
            df_pandas['direction_volatility'] = df_pandas.groupby(self.gcols)['dir'].transform(
                lambda x: x.rolling(3, min_periods=1).std()
            ).fillna(0.0)
        else:
            df_pandas['direction_volatility'] = 0.0
        
        # 4. Broken play score (combinación de presión + movimiento errático)
        df_pandas['broken_play_score'] = (
            df_pandas['pressure_intensity'] * 
            df_pandas['direction_volatility'] * 
            df_pandas['escape_velocity'].clip(0, 5)
        ).clip(0, 10).astype(np.float32)
        
        new_cols = ['pressure_intensity', 'escape_velocity', 'direction_volatility', 'broken_play_score']
        return df_pandas, new_cols
    
    def _create_ball_tracking_features(self, df_pandas):
        """Features centradas en la trayectoria del balón - CRÍTICO para BDB 2026"""
        new_cols = []
        
        # Verificar que existan columnas del punto de aterrizaje del balón
        if 'ball_land_x' not in df_pandas.columns or 'ball_land_y' not in df_pandas.columns:
            print("  [!] Skipping ball_tracking (ball_land_x/ball_land_y no disponibles)")
            return df_pandas, new_cols
        
        # 1. Distancia al punto de aterrizaje del balón (feature más importante)
        df_pandas['dist_to_ball'] = np.sqrt(
            (df_pandas['x'] - df_pandas['ball_land_x'])**2 + 
            (df_pandas['y'] - df_pandas['ball_land_y'])**2
        )
        
        # 2. Velocidad de acercamiento al balón (closing speed)
        df_pandas['closing_speed_to_ball'] = (
            df_pandas.groupby(self.gcols)['dist_to_ball'].diff().fillna(0.0) * -FPS
        )
        
        # 3. Ángulo hacia el punto de aterrizaje del balón
        df_pandas['angle_to_ball'] = np.arctan2(
            df_pandas['ball_land_y'] - df_pandas['y'],
            df_pandas['ball_land_x'] - df_pandas['x']
        ) * 180 / np.pi
        
        # 4. Desalineación con el balón (qué tan bien apunta hacia el balón)
        if 'dir' in df_pandas.columns:
            df_pandas['ball_alignment_error'] = np.abs(df_pandas['dir'] - df_pandas['angle_to_ball'])
            df_pandas['ball_alignment_error'] = np.minimum(
                df_pandas['ball_alignment_error'], 
                360 - df_pandas['ball_alignment_error']
            )
        else:
            df_pandas['ball_alignment_error'] = 0.0
        
        # 5. Tiempo estimado para llegar al balón
        if 'velocity_magnitude' in df_pandas.columns:
            df_pandas['time_to_ball'] = df_pandas['dist_to_ball'] / (
                df_pandas['velocity_magnitude'] + 0.1
            )
        else:
            df_pandas['time_to_ball'] = 999.0
        
        # 6. Aceleración hacia el punto de aterrizaje del balón (proyección vectorial)
        if 'acceleration_x' in df_pandas.columns and 'acceleration_y' in df_pandas.columns:
            df_pandas['accel_to_ball'] = (
                df_pandas['acceleration_x'] * (df_pandas['ball_land_x'] - df_pandas['x']) +
                df_pandas['acceleration_y'] * (df_pandas['ball_land_y'] - df_pandas['y'])
            ) / (df_pandas['dist_to_ball'] + 0.1)
        else:
            df_pandas['accel_to_ball'] = 0.0
        
        new_cols = [
            'dist_to_ball', 'closing_speed_to_ball', 'angle_to_ball',
            'ball_alignment_error', 'time_to_ball', 'accel_to_ball'
        ]
        return df_pandas, new_cols
    
    def _create_receiver_features(self, df_pandas):
        """Features específicas para receptores objetivo - ULTRA-OPTIMIZADO (vectorizado)"""
        new_cols = []
        
        if 'dist_to_ball' not in df_pandas.columns:
            print("  [!] Skipping receiver_features (requiere ball_tracking)")
            return df_pandas, new_cols
        
        # 1. Identificar receptor objetivo (vectorizado con groupby + idxmin)
        df_pandas['is_targeted_receiver'] = 0
        
        if 'is_offense' in df_pandas.columns:
            # Filtrar solo offense
            off_df = df_pandas[df_pandas['is_offense'] == 1].copy()
            if len(off_df) > 0:
                # Encontrar el más cercano por frame (vectorizado)
                closest_indices = off_df.groupby(['game_id', 'play_id', 'frame_id'])['dist_to_ball'].idxmin()
                df_pandas.loc[closest_indices, 'is_targeted_receiver'] = 1
        
        # 2. Ventaja del receptor (vectorizado con groupby + merge)
        df_pandas['receiver_advantage'] = 0.0
        
        if 'is_defense' in df_pandas.columns and (df_pandas['is_targeted_receiver'] == 1).sum() > 0:
            # Calcular distancia mínima de defensores por frame (vectorizado)
            def_min_dist = df_pandas[df_pandas['is_defense'] == 1].groupby(
                ['game_id', 'play_id', 'frame_id']
            )['dist_to_ball'].min().reset_index()
            def_min_dist.columns = ['game_id', 'play_id', 'frame_id', 'min_def_dist']
            
            # Merge con receptores
            rec_mask = df_pandas['is_targeted_receiver'] == 1
            rec_indices = df_pandas[rec_mask].index
            rec_df = df_pandas.loc[rec_indices, ['game_id', 'play_id', 'frame_id', 'dist_to_ball']].copy()
            rec_df = rec_df.merge(def_min_dist, on=['game_id', 'play_id', 'frame_id'], how='left')
            rec_df['advantage'] = rec_df['min_def_dist'].fillna(999) - rec_df['dist_to_ball']
            
            # Asignar de vuelta (vectorizado)
            df_pandas.loc[rec_indices, 'receiver_advantage'] = rec_df['advantage'].values
        
        # 3. Momentum (vectorizado - ya existe en columnas)
        if 'velocity_x' in df_pandas.columns and 'velocity_y' in df_pandas.columns:
            df_pandas['momentum_to_ball'] = (
                df_pandas['velocity_x'] * (df_pandas['ball_land_x'] - df_pandas['x']) +
                df_pandas['velocity_y'] * (df_pandas['ball_land_y'] - df_pandas['y'])
            ) / (df_pandas['dist_to_ball'] + 0.1)
        else:
            df_pandas['momentum_to_ball'] = 0.0
        
        new_cols = ['is_targeted_receiver', 'receiver_advantage', 'momentum_to_ball']
        return df_pandas, new_cols
    
    def _create_defender_competition_features(self, df_pandas):
        """Features para modelar competencia entre múltiples defensores - ULTRA-OPTIMIZADO (vectorizado)"""
        new_cols = []
        
        if 'dist_to_ball' not in df_pandas.columns or 'is_defense' not in df_pandas.columns:
            print("  [!] Skipping defender_competition (requiere ball_tracking y is_defense)")
            return df_pandas, new_cols
        
        # Inicializar columnas
        df_pandas['defender_rank_to_ball'] = 0
        df_pandas['num_defenders_closer'] = 0
        df_pandas['defensive_pressure_on_ball'] = 0.0
        
        # VECTORIZADO: Ranking de defensores por frame
        def_df = df_pandas[df_pandas['is_defense'] == 1].copy()
        if len(def_df) > 0:
            def_df['rank'] = def_df.groupby(['game_id', 'play_id', 'frame_id'])['dist_to_ball'].rank(method='min')
            def_df['num_closer'] = def_df['rank'] - 1
            
            # Asignar de vuelta (vectorizado con loc)
            df_pandas.loc[def_df.index, 'defender_rank_to_ball'] = def_df['rank'].values
            df_pandas.loc[def_df.index, 'num_defenders_closer'] = def_df['num_closer'].values
        
        # VECTORIZADO: Presión defensiva (cuántos defensores < 5 yardas)
        pressure_counts = df_pandas[
            (df_pandas['is_defense'] == 1) & (df_pandas['dist_to_ball'] < 5.0)
        ].groupby(['game_id', 'play_id', 'frame_id']).size().reset_index(name='pressure')
        
        # Merge y asignar
        df_pandas = df_pandas.merge(
            pressure_counts, 
            on=['game_id', 'play_id', 'frame_id'], 
            how='left'
        )
        df_pandas['defensive_pressure_on_ball'] = df_pandas['pressure'].fillna(0).astype(np.float32)
        df_pandas.drop('pressure', axis=1, inplace=True)
        
        new_cols = ['defender_rank_to_ball', 'num_defenders_closer', 'defensive_pressure_on_ball']
        return df_pandas, new_cols
    
    def _create_coverage_scheme_features(self, df_pandas):
        """Detecta esquemas de cobertura defensiva (Cover 2/3/4) - COMPLETAMENTE VECTORIZADO"""
        new_cols = []
        
        if 'is_defense' not in df_pandas.columns:
            print("  [!] Skipping coverage_scheme (requiere is_defense)")
            return df_pandas, new_cols
        
        # Inicializar columnas
        df_pandas['deep_defenders'] = 0
        df_pandas['coverage_type'] = 0
        df_pandas['coverage_depth'] = 0.0
        df_pandas['coverage_width'] = 0.0
        df_pandas['is_in_coverage_hole'] = 0
        
        # VECTORIZADO: Calcular todo con groupby + transform (sin loops)
        def_df = df_pandas[df_pandas['is_defense'] == 1].copy()
        
        if len(def_df) > 0:
            # 1. Calcular percentil 60 de 'y' por frame (vectorizado)
            def_df['y_p60'] = def_df.groupby(['game_id', 'play_id', 'frame_id'])['y'].transform(lambda x: np.percentile(x, 60))
            def_df['is_deep'] = (def_df['y'] > def_df['y_p60']).astype(int)
            
            # 2. Contar defensores profundos por frame
            deep_counts = def_df.groupby(['game_id', 'play_id', 'frame_id'])['is_deep'].sum().reset_index()
            deep_counts.columns = ['game_id', 'play_id', 'frame_id', 'num_deep']
            
            # 3. Determinar coverage type (vectorizado)
            deep_counts['coverage_type'] = 0
            deep_counts.loc[deep_counts['num_deep'] >= 4, 'coverage_type'] = 4
            deep_counts.loc[deep_counts['num_deep'] == 3, 'coverage_type'] = 3
            deep_counts.loc[deep_counts['num_deep'] == 2, 'coverage_type'] = 2
            
            # 4. Calcular depth y width (vectorizado)
            stats = def_df.groupby(['game_id', 'play_id', 'frame_id']).agg({
                'y': 'mean',
                'x': ['min', 'max']
            }).reset_index()
            stats.columns = ['game_id', 'play_id', 'frame_id', 'avg_depth', 'x_min', 'x_max']
            stats['coverage_width'] = stats['x_max'] - stats['x_min']
            
            # 5. Merge todo de una vez
            frame_stats = deep_counts.merge(
                stats[['game_id', 'play_id', 'frame_id', 'avg_depth', 'coverage_width']], 
                on=['game_id', 'play_id', 'frame_id']
            )
            
            # 6. Merge con DataFrame principal
            df_pandas = df_pandas.merge(
                frame_stats[['game_id', 'play_id', 'frame_id', 'num_deep', 'coverage_type', 'avg_depth', 'coverage_width']],
                on=['game_id', 'play_id', 'frame_id'],
                how='left',
                suffixes=('', '_cov')
            )
            
            # Reemplazar columnas
            df_pandas['deep_defenders'] = df_pandas['num_deep'].fillna(0).astype(int)
            df_pandas['coverage_type'] = df_pandas['coverage_type_cov'].fillna(0).astype(int)
            df_pandas['coverage_depth'] = df_pandas['avg_depth'].fillna(0.0)
            df_pandas['coverage_width'] = df_pandas['coverage_width_cov'].fillna(0.0)
            # BUG FIX: Proteger drop verificando existencia de columnas
            cols_to_drop = ['num_deep', 'coverage_type_cov', 'avg_depth', 'coverage_width_cov']
            existing_cols = [c for c in cols_to_drop if c in df_pandas.columns]
            if existing_cols:
                df_pandas.drop(existing_cols, axis=1, inplace=True)
            
            # 7. is_in_coverage_hole (vectorizado con máscaras)
            # Cover 2: flats laterales
            mask_c2 = (df_pandas['coverage_type'] == 2)
            df_pandas.loc[mask_c2 & ((df_pandas['x'] < 10) | (df_pandas['x'] > 43)), 'is_in_coverage_hole'] = 1
            
            # Cover 3: speed outs
            mask_c3 = (df_pandas['coverage_type'] == 3)
            df_pandas.loc[mask_c3 & ((df_pandas['x'] < 15) | (df_pandas['x'] > 38)), 'is_in_coverage_hole'] = 1
            
            # Cover 4: seams
            mask_c4 = (df_pandas['coverage_type'] == 4)
            df_pandas.loc[mask_c4 & (20 < df_pandas['x']) & (df_pandas['x'] < 33) & (20 < df_pandas['y']) & (df_pandas['y'] < 35), 'is_in_coverage_hole'] = 1
        
        # 6. Ventaja táctica del receptor según cobertura
        if 'is_targeted_receiver' in df_pandas.columns:
            # Cover 2: ventaja en rutas verticales (seams)
            df_pandas['cover2_advantage'] = (
                (df_pandas['coverage_type'] == 2) & 
                (df_pandas['is_targeted_receiver'] == 1) &
                (20 < df_pandas['x']) & (df_pandas['x'] < 33)  # En seam
            ).astype(np.int8)
            
            # Cover 3: ventaja en flats rápidos
            df_pandas['cover3_advantage'] = (
                (df_pandas['coverage_type'] == 3) & 
                (df_pandas['is_targeted_receiver'] == 1) &
                ((df_pandas['x'] < 15) | (df_pandas['x'] > 38))  # En flats
            ).astype(np.int8)
            
            # Cover 4: ventaja en rutas intermedias
            df_pandas['cover4_advantage'] = (
                (df_pandas['coverage_type'] == 4) & 
                (df_pandas['is_targeted_receiver'] == 1) &
                (15 < df_pandas['y']) & (df_pandas['y'] < 25)  # Zona intermedia
            ).astype(np.int8)
            
            new_cols.extend(['cover2_advantage', 'cover3_advantage', 'cover4_advantage'])
        
        new_cols = ['deep_defenders', 'coverage_type', 'coverage_depth', 
                    'coverage_width', 'is_in_coverage_hole'] + new_cols
        return df_pandas, new_cols
        
    def _create_player_interaction_distance_features(self, df_pandas):
        """
        ULTRA-OPTIMIZED: Patrón de acumulación como advanced_pressure
        - Vectorización NumPy (rápido)
        - Acumulación + 1 sola asignación por columna (bajo overhead)
        - Reduce de 1+ hora a ~30-60 segundos
        """
        new_cols = [
            'min_opponent_distance',
            'min_teammate_distance', 
            'relative_velocity_to_nearest_opponent',
            'opponent_density_5yd',
            'teammate_density_5yd'
        ]
        
        # Verificar que existan las columnas necesarias
        if not {'x', 'y', 'velocity_x', 'velocity_y', 'is_offense'}.issubset(df_pandas.columns):
            return df_pandas, []
        
        # Inicializar columnas con valores por defecto
        for col in new_cols:
            if 'distance' in col or 'velocity' in col:
                df_pandas[col] = 999.0  # Default para distancias
            else:
                df_pandas[col] = 0  # Default para densidades
        
        # Acumuladores (patrón de advanced_pressure)
        idx_accum = []
        min_opp_accum = []
        min_team_accum = []
        rel_vel_accum = []
        opp_dens_accum = []
        team_dens_accum = []
        
        # Loop por frame (mantener frame_id como solicitado)
        for (gid, pid, fid), frame_df in df_pandas.groupby(['game_id', 'play_id', 'frame_id']):
            # Trabajar con vista (NO .copy())
            indices = frame_df.index.to_numpy()
            
            # Extraer datos (sin conversión repetida de tipos)
            positions = frame_df[['x', 'y']].values
            velocities = frame_df[['velocity_x', 'velocity_y']].values
            is_offense = frame_df['is_offense'].values
            
            n_players = len(frame_df)
            
            if n_players < 2:
                continue  # Skip frames con muy pocos jugadores
            
            # === VECTORIZACIÓN COMPLETA ===
            
            # 1. Matriz de distancias (n × n)
            pos_diff = positions[:, np.newaxis, :] - positions[np.newaxis, :, :]
            distances = np.sqrt(np.sum(pos_diff**2, axis=2))
            np.fill_diagonal(distances, np.inf)
            
            # 2. Máscaras de oponentes/compañeros (n × n)
            opponent_masks = is_offense[:, np.newaxis] != is_offense[np.newaxis, :]
            teammate_masks = is_offense[:, np.newaxis] == is_offense[np.newaxis, :]
            np.fill_diagonal(opponent_masks, False)
            np.fill_diagonal(teammate_masks, False)
            
            # 3. Distancias enmascaradas
            dist_to_opponents = np.where(opponent_masks, distances, np.inf)
            dist_to_teammates = np.where(teammate_masks, distances, np.inf)
            
            # 4. Features vectorizadas
            min_opponent_dist = np.min(dist_to_opponents, axis=1).astype(np.float32)
            min_teammate_dist = np.min(dist_to_teammates, axis=1).astype(np.float32)
            opponent_density = np.sum((dist_to_opponents < 5.0) & (dist_to_opponents < np.inf), axis=1).astype(np.int8)
            teammate_density = np.sum((dist_to_teammates < 5.0) & (dist_to_teammates < np.inf), axis=1).astype(np.int8)
            
            # 5. Velocidad relativa (vectorizada cuando es posible)
            nearest_opponent_idx = np.argmin(dist_to_opponents, axis=1)
            within_20yd = min_opponent_dist < 20.0
            relative_velocity = np.zeros(n_players, dtype=np.float32)
            
            if within_20yd.any():
                valid_mask = within_20yd
                valid_idx = np.where(valid_mask)[0]
                
                for i in valid_idx:
                    nearest_opp = nearest_opponent_idx[i]
                    vel_diff = velocities[i] - velocities[nearest_opp]
                    pos_vec = positions[nearest_opp] - positions[i]
                    dist = distances[i, nearest_opp]
                    
                    if dist > 1e-6:
                        relative_velocity[i] = np.dot(vel_diff, pos_vec) / dist
            
            # Acumular resultados (NO asignar todavía)
            idx_accum.append(indices)
            min_opp_accum.append(min_opponent_dist)
            min_team_accum.append(min_teammate_dist)
            rel_vel_accum.append(relative_velocity)
            opp_dens_accum.append(opponent_density)
            team_dens_accum.append(teammate_density)
        
        # UNA SOLA asignación por columna al final (patrón de advanced_pressure)
        if idx_accum:
            idx_all = np.concatenate(idx_accum)
            df_pandas.loc[idx_all, 'min_opponent_distance'] = np.concatenate(min_opp_accum)
            df_pandas.loc[idx_all, 'min_teammate_distance'] = np.concatenate(min_team_accum)
            df_pandas.loc[idx_all, 'relative_velocity_to_nearest_opponent'] = np.concatenate(rel_vel_accum)
            df_pandas.loc[idx_all, 'opponent_density_5yd'] = np.concatenate(opp_dens_accum)
            df_pandas.loc[idx_all, 'teammate_density_5yd'] = np.concatenate(team_dens_accum)
        
        return df_pandas, new_cols

    def transform(self, df):
        # Trabajar 100% en Pandas (sin conversiones Polars↔Pandas)
        if isinstance(df, pl.DataFrame):
            df = df.to_pandas()
        elif isinstance(df, pd.DataFrame):
            df = df.copy()
        else:
            # Si es array numpy u otro tipo, intentar convertir
            df = pd.DataFrame(df)
        
        # Sort by grouping columns and frame_id
        df = df.sort_values(['game_id','play_id','nfl_id','frame_id'])
        
        # Process basic features (all in Pandas)
        df = self._create_basic_features_pandas(df)

        print("\nStep 2/3: Adding selected advanced features...")
        for group_name in self.active_groups:
            if group_name in self.feature_creators:
                t0 = time.perf_counter()
                creator = self.feature_creators[group_name]
                df, new_cols = creator(df)
                dt = time.perf_counter() - t0
                self.created_feature_cols.extend(new_cols)
                print(f"  [+] Added '{group_name:<22}' ({len(new_cols):2} cols) in {dt*1000:7.1f} ms")
            else:
                print(f"  [!] Unknown feature group: {group_name}")

        final_cols = sorted(set(self.created_feature_cols))
        print(f"\nTotal features created: {len(final_cols)}")
        return df, final_cols

# -------------------------------
# Model & training (same spirit as your version)
# -------------------------------
class TemporalHuber(nn.Module):
    def __init__(self, delta=0.5, time_decay=0.03, velocity_penalty_weight=0.01,
                 acceleration_penalty_weight=0.0, use_huber_for_penalty=True):
        """
        参数:
        delta: Huber损失的阈值
        time_decay: 时间衰减系数，越大则未来时刻权重越小
        velocity_penalty_weight: 速度变化惩罚权重（一阶差分）
        acceleration_penalty_weight: 加速度惩罚权重（二阶差分）
        use_huber_for_penalty: 是否对正则项也使用Huber损失
        """
        super().__init__()
        self.delta = delta
        self.time_decay = time_decay
        self.velocity_penalty_weight = velocity_penalty_weight
        self.acceleration_penalty_weight = acceleration_penalty_weight
        self.use_huber_for_penalty = use_huber_for_penalty
    
    def forward(self, pred, target, mask):
        err = pred - target
        abs_err = torch.abs(err)
        
        # ===== 主Huber损失 =====
        huber = torch.where(abs_err <= self.delta,
                           0.5 * err * err,
                           self.delta * (abs_err - 0.5 * self.delta))
        
        # 时间衰减权重
        if self.time_decay > 0:
            L = pred.size(1)
            t = torch.arange(L, device=pred.device).float()
            w = torch.exp(-self.time_decay * t).view(1, L)
            huber = huber * w
            mask_weighted = mask * w
        else:
            mask_weighted = mask
        
        main_loss = (huber * mask_weighted).sum() / (mask_weighted.sum() + 1e-8)
        
        # ===== 速度平滑正则项 =====
        velocity_penalty = 0.0
        if self.velocity_penalty_weight > 0 and pred.size(1) > 1:
            # 一阶差分（速度变化）
            velocity_diff = pred[:, 1:] - pred[:, :-1]
            mask_vel = mask[:, 1:]
            
            if self.use_huber_for_penalty:
                vel_abs = torch.abs(velocity_diff)
                vel_loss = torch.where(vel_abs <= self.delta,
                                      0.5 * velocity_diff * velocity_diff,
                                      self.delta * (vel_abs - 0.5 * self.delta))
            else:
                vel_loss = velocity_diff * velocity_diff
            
            # 应用时间衰减（可选）
            if self.time_decay > 0:
                L_vel = vel_loss.size(1)
                t_vel = torch.arange(L_vel, device=pred.device).float()
                w_vel = torch.exp(-self.time_decay * t_vel).view(1, L_vel)
                vel_loss = vel_loss * w_vel
                mask_vel = mask_vel * w_vel
            
            velocity_penalty = (vel_loss * mask_vel).sum() / (mask_vel.sum() + 1e-8)
        
        # ===== 加速度平滑正则项 =====
        acceleration_penalty = 0.0
        if self.acceleration_penalty_weight > 0 and pred.size(1) > 2:
            # 二阶差分（加速度变化）
            velocity_diff = pred[:, 1:] - pred[:, :-1]
            acceleration = velocity_diff[:, 1:] - velocity_diff[:, :-1]
            mask_acc = mask[:, 2:]
            
            if self.use_huber_for_penalty:
                acc_abs = torch.abs(acceleration)
                acc_loss = torch.where(acc_abs <= self.delta,
                                      0.5 * acceleration * acceleration,
                                      self.delta * (acc_abs - 0.5 * self.delta))
            else:
                acc_loss = acceleration * acceleration
            
            # 应用时间衰减（可选）
            if self.time_decay > 0:
                L_acc = acc_loss.size(1)
                t_acc = torch.arange(L_acc, device=pred.device).float()
                w_acc = torch.exp(-self.time_decay * t_acc).view(1, L_acc)
                acc_loss = acc_loss * w_acc
                mask_acc = mask_acc * w_acc
            
            acceleration_penalty = (acc_loss * mask_acc).sum() / (mask_acc.sum() + 1e-8)
        
        # ===== 组合损失 =====
        total_loss = (main_loss +
                     self.velocity_penalty_weight * velocity_penalty +
                     self.acceleration_penalty_weight * acceleration_penalty)
        
        return total_loss

class UncertaintyLoss(nn.Module):
    """Negative Log-Likelihood loss para predicción con incertidumbre"""
    def __init__(self, delta=0.5, time_decay=0.03):
        super().__init__()
        self.delta = delta
        self.time_decay = time_decay
    
    def forward(self, pred_mean, pred_logvar, target, mask, chaos_weight=None):
        # pred_mean, pred_logvar, target: (B, horizon)
        # mask: (B, horizon)
        # chaos_weight: (B, horizon) - peso adicional para jugadas caóticas (1.0 normal, 1.5 para caos)
        
        # Calcular precision (inverso de varianza)
        precision = torch.exp(-pred_logvar)
        
        # Huber loss ponderado por precision
        diff = pred_mean - target
        abs_diff = torch.abs(diff)
        
        # Huber: cuadrático si |diff| < delta, lineal si >= delta
        huber = torch.where(
            abs_diff < self.delta,
            0.5 * diff ** 2,
            self.delta * (abs_diff - 0.5 * self.delta)
        )
        
        # NLL: precision * huber + logvar (penaliza baja certeza con error alto)
        nll = precision * huber + pred_logvar
        
        # Aplicar peso de caos si está disponible (1.0 normal, 1.5 para caos)
        if chaos_weight is not None:
            nll = nll * chaos_weight
        
        # Decay temporal (más peso a predicciones cercanas)
        H = target.shape[1]
        t = torch.arange(H, device=target.device, dtype=target.dtype)
        decay = torch.exp(-self.time_decay * t).view(1, H)
        
        # Aplicar mask y decay
        loss = (nll * mask * decay).sum() / (mask.sum() + 1e-8)
        
        return loss

class SeqModel(nn.Module):
    def __init__(self, input_dim, horizon):
        super().__init__()
        # 投影到可被num_heads整除的维度
        self.hidden_dim = 128
        self.input_proj = nn.Linear(input_dim, self.hidden_dim)
        
        # 位置编码（假设序列长度最大为10）
        self.pos_encoding = nn.Parameter(torch.randn(1, 10, self.hidden_dim) * 0.02)
        
        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.hidden_dim,
            nhead=4,
            dim_feedforward=256,
            dropout=0.1,
            activation='gelu',
            batch_first=True,
            norm_first=True  # Pre-LN更稳定
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)
        
        # Pooling层
        self.pool_ln = nn.LayerNorm(self.hidden_dim)
        self.pool_attn = nn.MultiheadAttention(
            self.hidden_dim, 
            num_heads=4, 
            batch_first=True,
            dropout=0.1
        )
        self.pool_query = nn.Parameter(torch.randn(1, 1, self.hidden_dim))
        
        # 输出头 - 使用PReLU替代GELU
        self.head = nn.Sequential(
            nn.Linear(self.hidden_dim, self.hidden_dim),
            nn.PReLU(num_parameters=self.hidden_dim),  # 每个通道独立学习负斜率
            nn.Dropout(0.2),
            nn.Linear(self.hidden_dim, horizon)
        )
    
    def forward(self, x):
        # x: (B, seq_len, input_dim)
        B, seq_len, _ = x.shape
        
        # 投影输入
        x = self.input_proj(x)  # (B, seq_len, hidden_dim)
        
        # 添加位置编码
        x = x + self.pos_encoding[:, :seq_len, :]
        
        # Transformer编码
        h = self.transformer(x)  # (B, seq_len, hidden_dim)
        
        # 注意力池化
        q = self.pool_query.expand(B, -1, -1)  # (B, 1, hidden_dim)
        h_norm = self.pool_ln(h)  # (B, seq_len, hidden_dim)
        ctx, _ = self.pool_attn(q, h_norm, h_norm)  # (B, 1, hidden_dim)
        
        # 预测
        out = self.head(ctx.squeeze(1))  # (B, horizon)
        
        # 累积和
        return torch.cumsum(out, dim=1)  # (B, horizon)

class SeqModelWithUncertainty(nn.Module):
    """Modelo que predice posición (mean) + incertidumbre (logvar)"""
    def __init__(self, input_dim, horizon):
        super().__init__()
        # Reutilizar arquitectura base de SeqModel
        self.hidden_dim = 128
        self.input_proj = nn.Linear(input_dim, self.hidden_dim)
        self.pos_encoding = nn.Parameter(torch.randn(1, 10, self.hidden_dim) * 0.02)
        
        # Transformer Encoder (igual que SeqModel)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.hidden_dim, nhead=4, dim_feedforward=256,
            dropout=0.1, activation='gelu', batch_first=True, norm_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)
        
        # Pooling con attention (igual que SeqModel)
        self.pool_ln = nn.LayerNorm(self.hidden_dim)
        self.pool_attn = nn.MultiheadAttention(
            self.hidden_dim, num_heads=4, batch_first=True, dropout=0.1
        )
        self.pool_query = nn.Parameter(torch.randn(1, 1, self.hidden_dim))
        
        # NUEVO: Cabezas separadas para mean y logvar
        self.mean_head = nn.Sequential(
            nn.Linear(self.hidden_dim, self.hidden_dim),
            nn.PReLU(num_parameters=self.hidden_dim),
            nn.Dropout(0.2),
            nn.Linear(self.hidden_dim, horizon)
        )
        
        self.logvar_head = nn.Sequential(
            nn.Linear(self.hidden_dim, self.hidden_dim // 2),
            nn.PReLU(num_parameters=self.hidden_dim // 2),
            nn.Dropout(0.1),
            nn.Linear(self.hidden_dim // 2, horizon)
        )
    
    def forward(self, x):
        # x: (B, seq_len, input_dim)
        B, seq_len, _ = x.shape
        
        # Encoding (igual que SeqModel)
        x = self.input_proj(x)
        x = x + self.pos_encoding[:, :seq_len, :]
        h = self.transformer(x)
        
        # Pooling con attention
        q = self.pool_query.expand(B, -1, -1)
        h_norm = self.pool_ln(h)
        ctx, _ = self.pool_attn(q, h_norm, h_norm)
        features = ctx.squeeze(1)  # (B, hidden_dim)
        
        # Predicciones separadas
        mean = self.mean_head(features)  # (B, horizon)
        logvar = self.logvar_head(features)  # (B, horizon)
        
        # Cumsum solo en mean
        mean_cumsum = torch.cumsum(mean, dim=1)
        
        return mean_cumsum, logvar

def prepare_targets(batch_axis, max_h):
    tensors, masks = [], []
    for arr in batch_axis:
        L = len(arr)
        padded = np.pad(arr, (0, max_h - L), constant_values=0).astype(np.float32)
        mask = np.zeros(max_h, dtype=np.float32)
        mask[:L] = 1.0
        tensors.append(torch.tensor(padded))
        masks.append(torch.tensor(mask))
    return torch.stack(tensors), torch.stack(masks)

def train_model(X_train, y_train, X_val, y_val, input_dim, horizon, config, use_uncertainty=False, feat_cols=None):
    device = config.DEVICE
    
    # Seleccionar modelo y loss
    if use_uncertainty:
        model = SeqModelWithUncertainty(input_dim, horizon).to(device)
        criterion = UncertaintyLoss(delta=0.5, time_decay=0.03)
    else:
        model = SeqModel(input_dim, horizon).to(device)
        criterion = TemporalHuber(delta=0.5, time_decay=0.03)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.LEARNING_RATE, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5, verbose=False)

    # build batches (keep numpy → torch)
    def build_batches(X, Y):
        batches = []
        B = config.BATCH_SIZE
        for i in range(0, len(X), B):
            end = min(i + B, len(X))
            xs = torch.tensor(np.stack(X[i:end]).astype(np.float32))
            ys, ms = prepare_targets([Y[j] for j in range(i, end)], horizon)
            batches.append((xs, ys, ms))
        return batches

    tr_batches = build_batches(X_train, y_train)
    va_batches = build_batches(X_val,   y_val)

    best_loss, best_state, bad = float('inf'), None, 0
    for epoch in range(1, config.EPOCHS + 1):
        model.train()
        train_losses = []
        for bx, by, bm in tr_batches:
            bx, by, bm = bx.to(device), by.to(device), bm.to(device)
            
            if use_uncertainty:
                pred_mean, pred_logvar = model(bx)
                
                # Combinar proximidad al balón (70%) + caos (30%) para weighted loss
                combined_weight = None
                if feat_cols is not None:
                    weight = torch.ones_like(bx[:, -1, 0])  # Base 1.0, shape (B,)
                    
                    # Factor 1: Proximidad al balón (MÁS IMPORTANTE - 70% del peso adicional)
                    if 'dist_to_ball' in feat_cols:
                        ball_dist_idx = feat_cols.index('dist_to_ball')
                        ball_distances = bx[:, -1, ball_dist_idx]
                        # Invertir distancia: cerca del balón = más peso
                        # dist_to_ball típicamente en [0, 50+]
                        proximity_ratio = 10.0 / (ball_distances + 1.0)
                        proximity_factor = torch.clamp(proximity_ratio, 0, 0.35)  # 0 a 0.35
                        weight = weight + proximity_factor
                    
                    # Factor 2: Caos (MENOS IMPORTANTE - 30% del peso adicional)
                    if 'broken_play_score' in feat_cols:
                        chaos_idx = feat_cols.index('broken_play_score')
                        chaos_scores = bx[:, -1, chaos_idx]
                        # broken_play_score está en [0, 10]
                        chaos_factor = (chaos_scores / 10.0) * 0.15  # 0 a 0.15
                        weight = weight + chaos_factor
                    
                    # Factor 3: Coverage Hole (5% adicional - BONUS para jugadas en huecos de cobertura)
                    if 'is_in_coverage_hole' in feat_cols:
                        hole_idx = feat_cols.index('is_in_coverage_hole')
                        hole_flags = bx[:, -1, hole_idx]
                        # is_in_coverage_hole es binario (0 o 1)
                        hole_factor = hole_flags * 0.05  # 0 o 0.05
                        weight = weight + hole_factor
                    
                    # Expandir a (B, horizon) para aplicar a todas las predicciones
                    combined_weight = weight.unsqueeze(1).expand(-1, horizon)
                
                loss = criterion(pred_mean, pred_logvar, by, bm, combined_weight)
            else:
                pred = model(bx)
                loss = criterion(pred, by, bm)
            
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_losses.append(loss.item())

        model.eval()
        val_losses = []
        with torch.no_grad():
            for bx, by, bm in va_batches:
                bx, by, bm = bx.to(device), by.to(device), bm.to(device)
                
                if use_uncertainty:
                    pred_mean, pred_logvar = model(bx)
                    # También aplicar combined_weight en validación para consistencia
                    combined_weight = None
                    if feat_cols is not None:
                        weight = torch.ones_like(bx[:, -1, 0])
                        
                        # Factor 1: Proximidad al balón (70%)
                        if 'dist_to_ball' in feat_cols:
                            ball_dist_idx = feat_cols.index('dist_to_ball')
                            ball_distances = bx[:, -1, ball_dist_idx]
                            proximity_ratio = 10.0 / (ball_distances + 1.0)
                            proximity_factor = torch.clamp(proximity_ratio, 0, 0.35)
                            weight = weight + proximity_factor
                        
                        # Factor 2: Caos (30%)
                        if 'broken_play_score' in feat_cols:
                            chaos_idx = feat_cols.index('broken_play_score')
                            chaos_scores = bx[:, -1, chaos_idx]
                            chaos_factor = (chaos_scores / 10.0) * 0.15
                            weight = weight + chaos_factor
                        
                        # Factor 3: Coverage Hole (5% adicional)
                        if 'is_in_coverage_hole' in feat_cols:
                            hole_idx = feat_cols.index('is_in_coverage_hole')
                            hole_flags = bx[:, -1, hole_idx]
                            hole_factor = hole_flags * 0.05
                            weight = weight + hole_factor
                        
                        combined_weight = weight.unsqueeze(1).expand(-1, horizon)
                    
                    val_losses.append(criterion(pred_mean, pred_logvar, by, bm, combined_weight).item())
                else:
                    pred = model(bx)
                    val_losses.append(criterion(pred, by, bm).item())

        trl, val = float(np.mean(train_losses)), float(np.mean(val_losses))
        scheduler.step(val)
        if epoch % 10 == 0:
            print(f"  Epoch {epoch}: train={trl:.4f}, val={val:.4f}")

        if val < best_loss:
            best_loss, bad = val, 0
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
        else:
            bad += 1
            if bad >= config.PATIENCE:
                print(f"  Early stop at epoch {epoch}")
                break

    if best_state:
        model.load_state_dict(best_state)
    return model, best_loss

# ------------------------------_
# Main pipeline (MODIFICADO PARA ENSEMBLE DE SEMILLAS)
# ------------------------------_
def predict(test: pl.DataFrame, test_input: pl.DataFrame) -> pl.DataFrame:
    """Kaggle API predict function - optimized for speed with Polars and ModelRegistry"""
    print(f"Predicting for {len(test)} samples...")
    
    # Ensure inputs are Polars DataFrames
    if isinstance(test, pd.DataFrame):
        test = pandas_to_polars_optimized(test)
    if isinstance(test_input, pd.DataFrame):
        test_input = pandas_to_polars_optimized(test_input)
    
    # BUG FIX: Detectar automáticamente qué seeds están disponibles usando Config.SEEDS
    cfg = Config()
    available_seeds = [seed for seed in cfg.SEEDS if _model_registry.has_models(seed)]
    
    if not available_seeds:
        print("=" * 60)
        print("ENTRENAMIENTO INICIAL (primera llamada a predict)")
        print("Esto puede tardar varias horas - sin límite de tiempo")
        print("=" * 60)
        main()  # Entrena y cachea en _model_registry
        
        # Detectar seeds entrenados después del entrenamiento usando Config.SEEDS
        available_seeds = [seed for seed in cfg.SEEDS if _model_registry.has_models(seed)]
    
    if not available_seeds:
        print("❌ ERROR: No models available after training")
        print("Returning fallback predictions (all zeros)")
        return pl.DataFrame({'x': [0.0] * len(test), 'y': [0.0] * len(test)})
    
    # FIX #1: Usar TODOS los seeds disponibles para ensemble completo
    print(f"✓ Available seeds for ensemble: {available_seeds}")
    print(f"✓ Using multi-seed ensemble for maximum diversity")
    
    # Obtener metadata de la primera semilla para device y feature_cols
    first_seed = available_seeds[0]
    metadata = _model_registry.get_metadata(first_seed)
    trained_device = metadata.get('device', 'cpu')
    print(f"✓ Models trained on device: {trained_device}")
    
    # Prepare test sequences (already optimized for Polars)
    test_seqs, test_meta, feat_cols_t, dir_map_test = prepare_sequences_vectorized(
        test_input, test_template=test, is_training=False,
        window_size=Config.WINDOW_SIZE  # Consistent with training
    )
    
    # Get feature indices from metadata
    feat_cols = metadata.get('feature_columns', [])
    if not feat_cols:
        print("Error: No feature columns in metadata")
        return pl.DataFrame({'x': [0.0] * len(test), 'y': [0.0] * len(test)})
    
    idx_x = feat_cols.index('x')
    idx_y = feat_cols.index('y')
    
    # Prepare test data
    X_test_raw = list(test_seqs)
    x_last_uni = np.array([s[-1, idx_x] for s in X_test_raw], dtype=np.float32)
    y_last_uni = np.array([s[-1, idx_y] for s in X_test_raw], dtype=np.float32)
    
    # FIX #1: Ensemble multi-seed para máxima diversidad
    all_seed_preds_dx, all_seed_preds_dy = [], []
    
    # Get config from registry metadata
    cfg = Config()
    
    # Iterar sobre TODAS las semillas disponibles
    for seed in available_seeds:
        print(f"  Processing seed {seed}...")
        seed_models = _model_registry.get_best_models(seed)
        seed_scalers = _model_registry.get_best_scalers(seed)
        
        seed_preds_dx, seed_preds_dy = [], []
        
        for mx, my, sc in zip(seed_models['x'], seed_models['y'], seed_scalers):
            X_sc = np.stack([sc.transform(s) for s in X_test_raw]).astype(np.float32)
            X_t = torch.tensor(X_sc, dtype=torch.float32).to(trained_device)
            mx.eval()
            my.eval()
            with torch.no_grad():
                # Handle uncertainty models
                if cfg.USE_UNCERTAINTY:
                    pred_x, _ = mx(X_t)  # Ignorar logvar
                    pred_y, _ = my(X_t)
                    seed_preds_dx.append(pred_x.cpu().numpy())
                    seed_preds_dy.append(pred_y.cpu().numpy())
                else:
                    seed_preds_dx.append(mx(X_t).cpu().numpy())
                    seed_preds_dy.append(my(X_t).cpu().numpy())
        
        # Promediar modelos de esta semilla
        seed_avg_dx = np.mean(seed_preds_dx, axis=0)
        seed_avg_dy = np.mean(seed_preds_dy, axis=0)
        all_seed_preds_dx.append(seed_avg_dx)
        all_seed_preds_dy.append(seed_avg_dy)
    
    # FIX #1: Promediar TODAS las semillas (ensemble completo)
    ens_dx = np.mean(all_seed_preds_dx, axis=0)  # Shape: (n_sequences, horizon)
    ens_dy = np.mean(all_seed_preds_dy, axis=0)
    print(f"✓ Multi-seed ensemble completed: {len(available_seeds)} seeds × {len(seed_models['x'])} models")
    
    # Build predictions for EACH frame in test (not just one per sequence)
    predictions_list = []
    
    for i, meta in enumerate(test_meta):
        # Get metadata for this sequence
        game_id = meta['game_id']
        play_id = meta['play_id']
        nfl_id = meta['nfl_id']
        play_direction = meta['play_direction']
        
        # Filter test to get ALL future frames for this player
        target_frames = test.filter(
            (pl.col('game_id') == game_id) & 
            (pl.col('play_id') == play_id) & 
            (pl.col('nfl_id') == nfl_id)
        )
        
        # Get last known position from sequence
        last_x = x_last_uni[i]
        last_y = y_last_uni[i]
        
        # Create one prediction per future frame
        n_frames = len(target_frames)
        max_horizon = ens_dx.shape[1]
        
        for t in range(n_frames):
            # FIX #5: Interpolación de horizontes para evitar duplicación
            if t < max_horizon:
                # Usar horizonte exacto si está disponible
                horizon_idx = t
                dx_val = ens_dx[i, horizon_idx]
                dy_val = ens_dy[i, horizon_idx]
            else:
                # Interpolar linealmente entre los últimos 2 horizontes
                alpha = (t - (max_horizon - 1)) / max_horizon
                dx_val = (1 - alpha) * ens_dx[i, -2] + alpha * ens_dx[i, -1]
                dy_val = (1 - alpha) * ens_dy[i, -2] + alpha * ens_dy[i, -1]
            
            # Compute prediction
            pred_x_unified = last_x + dx_val
            pred_y_unified = last_y + dy_val
            
            # Apply field boundaries
            pred_x_unified = np.clip(pred_x_unified, 0.0, 120.0)
            pred_y_unified = np.clip(pred_y_unified, 0.0, 53.3)
            
            # Convert back to original direction
            if play_direction == 'right':
                pred_x_final = 120.0 - pred_x_unified
                pred_y_final = 53.3 - pred_y_unified
            else:
                pred_x_final = pred_x_unified
                pred_y_final = pred_y_unified
            
            predictions_list.append({
                'x': float(pred_x_final),
                'y': float(pred_y_final)
            })
    
    # Convert to Polars DataFrame
    return pl.DataFrame(predictions_list)

def main():
    # Limpiar registry al inicio para evitar state pollution (Issue #2)
    _model_registry.clear()
    
    cfg = Config()
    print("="*80)
    print(f"PASO 2: PIPELINE MEJORADO CON ENSEMBLE DE {len(cfg.SEEDS)} SEMILLAS + TTA SELECTIVO")
    print("="*80)
    print(f"Semillas a utilizar: {cfg.SEEDS}")
    print(f"Polars backend activo: True")

    # [1/4] Carga de datos (se hace una sola vez)
    print("\n[1/4] Cargando datos...")
    train_input_files  = [cfg.DATA_DIR / f"train/input_2023_w{w:02d}.csv"  for w in range(1, 19)]
    train_output_files = [cfg.DATA_DIR / f"train/output_2023_w{w:02d}.csv" for w in range(1, 19)]
    
    # Load with Polars for better performance - OPTIMIZED for memory
    # Cargar y concatenar directamente sin mantener listas intermedias
    train_input = pl.concat([pl.read_csv(f) for f in train_input_files if f.exists()])
    train_output = pl.concat([pl.read_csv(f) for f in train_output_files if f.exists()])
    test_input = pl.read_csv(cfg.DATA_DIR / "test_input.csv")
    test_template = pl.read_csv(cfg.DATA_DIR / "test.csv")

    # OPTIMIZACIÓN MEMORIA: Reducir precisión de float64 a float32 (50% menos memoria)
    print("✓ Optimizando tipos de datos para memoria...")
    float_cols = [col for col in train_input.columns if train_input[col].dtype == pl.Float64]
    for col in float_cols:
        train_input = train_input.with_columns(pl.col(col).cast(pl.Float32))
    
    float_cols_out = [col for col in train_output.columns if train_output[col].dtype == pl.Float64]
    for col in float_cols_out:
        train_output = train_output.with_columns(pl.col(col).cast(pl.Float32))
    
    float_cols_test = [col for col in test_input.columns if test_input[col].dtype == pl.Float64]
    for col in float_cols_test:
        test_input = test_input.with_columns(pl.col(col).cast(pl.Float32))
    
    print(f"✓ Convertidas {len(float_cols)} + {len(float_cols_out)} + {len(float_cols_test)} columnas a float32")

    # [2/4] Preparación de secuencias (se hace una sola vez)
    print("\n[2/4] Construyendo secuencias con características AVANZADAS...")
    seqs, tdx, tdy, seq_meta, feat_cols, dir_map = prepare_sequences_vectorized(
        train_input, output_df=train_output, is_training=True,
        window_size=cfg.WINDOW_SIZE
    )

    # Liberar DataFrames originales después de crear secuencias (CRÍTICO para memoria)
    del train_input, train_output
    gc.collect()
    print("✓ DataFrames originales liberados de memoria")

    # OPTIMIZACIÓN MEMORIA: Mantener como NumPy arrays (mucho más eficiente en memoria)
    sequences = seqs  # Ya es array NumPy
    targets_dx = tdx  # Ya es array NumPy
    targets_dy = tdy  # Ya es array NumPy
    print("✓ Secuencias mantenidas como NumPy arrays (optimización memoria)")

    # [3/4] Entrenamiento con GroupKFold sobre múltiples semillas
    print("\n[3/4] Iniciando entrenamiento de ensemble...")
    
    # Usar SOLO seed_models, eliminar all_models_x/y/scalers redundantes (OPTIMIZACIÓN MEMORIA)
    fold_rmse_list = []  # Para almacenar RMSE de cada fold
    
    # Diccionario para rastrear modelos por semilla
    seed_models = {}  # {seed: {'models_x': [], 'models_y': [], 'scalers': [], 'rmse_list': []}}
    
    groups = np.array([d['game_id'] for d in seq_meta])
    
    for seed in cfg.SEEDS:
        print(f"\n{'='*70}\n   Entrenando con Semilla (Seed): {seed}\n{'='*70}")
        set_seed(seed)
        
        # Inicializar contenedores para esta semilla
        seed_models[seed] = {
            'models_x': [],
            'models_y': [],
            'scalers': [],
            'rmse_list': []
        }
        
        gkf = GroupKFold(n_splits=cfg.N_FOLDS)

        for fold, (tr, va) in enumerate(gkf.split(sequences, groups=groups), 1):
            print(f"\n{'-'*60}\nFold {fold}/{cfg.N_FOLDS} para la semilla {seed}\n{'-'*60}")
            
            # Usar misma seed para todos los folds (consistencia)
            set_seed(seed)

            X_tr = [sequences[i] for i in tr]
            X_va = [sequences[i] for i in va]

            # Estandarización por fold
            scaler = StandardScaler()
            scaler.fit(np.vstack([s for s in X_tr]))

            X_tr_sc = np.stack([scaler.transform(s) for s in X_tr]).astype(np.float32)
            X_va_sc = np.stack([scaler.transform(s) for s in X_va]).astype(np.float32)

            # Entrenar modelo para X
            print("Entrenando modelo ΔX...")
            mx, loss_x = train_model(
                X_tr_sc, [targets_dx[i] for i in tr],
                X_va_sc, [targets_dx[i] for i in va],
                X_tr_sc.shape[-1], cfg.MAX_FUTURE_HORIZON, cfg,
                use_uncertainty=cfg.USE_UNCERTAINTY,
                feat_cols=feat_cols  # Pasar feature names para chaos_weight
            )

            # Entrenar modelo para Y
            print("Entrenando modelo ΔY...")
            my, loss_y = train_model(
                X_tr_sc, [targets_dy[i] for i in tr],
                X_va_sc, [targets_dy[i] for i in va],
                X_tr_sc.shape[-1], cfg.MAX_FUTURE_HORIZON, cfg,
                use_uncertainty=cfg.USE_UNCERTAINTY,
                feat_cols=feat_cols  # Pasar feature names para chaos_weight
            )
            
            # Guardar los modelos y el escalador de este fold (SOLO en seed_models)
            seed_models[seed]['models_x'].append(mx)
            seed_models[seed]['models_y'].append(my)
            seed_models[seed]['scalers'].append(scaler)
            
            # Calcular RMSE en el conjunto de validación
            mx.eval()
            my.eval()
            with torch.no_grad():
                X_va_t = torch.tensor(X_va_sc).to(cfg.DEVICE)
                
                # Handle uncertainty models
                if cfg.USE_UNCERTAINTY:
                    pred_dx, _ = mx(X_va_t)
                    pred_dy, _ = my(X_va_t)
                    pred_dx = pred_dx.cpu().numpy()
                    pred_dy = pred_dy.cpu().numpy()
                else:
                    pred_dx = mx(X_va_t).cpu().numpy()
                    pred_dy = my(X_va_t).cpu().numpy()
            
            # Preparar targets de validación para RMSE
            y_va_dx = [targets_dx[i] for i in va]
            y_va_dy = [targets_dy[i] for i in va]
            
            # Calcular RMSE: sqrt(mean((x_pred - x_true)^2 + (y_pred - y_true)^2))
            squared_errors = []
            for i in range(len(pred_dx)):
                # Obtener targets reales con padding
                target_dx_full, mask_dx = prepare_targets([y_va_dx[i]], cfg.MAX_FUTURE_HORIZON)
                target_dy_full, mask_dy = prepare_targets([y_va_dy[i]], cfg.MAX_FUTURE_HORIZON)
                
                target_dx_arr = target_dx_full[0].cpu().numpy()
                target_dy_arr = target_dy_full[0].cpu().numpy()
                mask_arr = mask_dx[0].cpu().numpy()
                
                # Solo calcular error en posiciones válidas (mask == 1)
                valid_indices = mask_arr > 0
                if valid_indices.sum() > 0:
                    dx_error = (pred_dx[i][valid_indices] - target_dx_arr[valid_indices]) ** 2
                    dy_error = (pred_dy[i][valid_indices] - target_dy_arr[valid_indices]) ** 2
                    # Sumar errores x e y para cada punto según fórmula oficial de Kaggle
                    point_errors = dx_error + dy_error
                    squared_errors.extend(point_errors)
            
            # Aplicar fórmula oficial: sqrt(mean / 2)
            fold_rmse = np.sqrt(np.mean(squared_errors) / 2.0)
            fold_rmse_list.append(fold_rmse)
            seed_models[seed]['rmse_list'].append(fold_rmse)
            
            print(f"Fold {fold} (semilla {seed}) — val loss: dx={loss_x:.5f}, dy={loss_y:.5f} | RMSE={fold_rmse:.5f}")
            
            # Liberar datos de entrenamiento del fold anterior (CRÍTICO para memoria)
            del X_tr, X_va, X_tr_sc, X_va_sc
            if 'pred_dx' in locals():
                del pred_dx, pred_dy
            if 'y_va_dx' in locals():
                del y_va_dx, y_va_dy
            gc.collect()

    # Calcular RMSE promedio por semilla e identificar la mejor
    print("\n" + "="*80)
    print("ESTADÍSTICAS DE RMSE POR SEMILLA")
    print("="*80)
    seed_avg_rmse = {}
    for seed in cfg.SEEDS:
        avg_rmse = np.mean(seed_models[seed]['rmse_list'])
        std_rmse = np.std(seed_models[seed]['rmse_list'])
        seed_avg_rmse[seed] = avg_rmse
        print(f"Semilla {seed}: RMSE Promedio = {avg_rmse:.5f} ± {std_rmse:.5f}")
    
    best_seed = min(seed_avg_rmse, key=seed_avg_rmse.get)
    print("-"*80)
    print(f"✓ MEJOR SEMILLA: {best_seed} con RMSE = {seed_avg_rmse[best_seed]:.5f}")
    print("  (Se aplicará TTA solo a los modelos de esta semilla)")
    print("="*80)
    
    # Guardar modelos entrenados para la API de Kaggle (OPTIMIZADO para memoria)
    # Usar ModelRegistry singleton en lugar de variables globales
    metadata = {
        'feature_columns': feat_cols,
        'sequence_meta': seq_meta,
        'direction_map': dir_map,
        'rmse_stats': seed_avg_rmse
    }
    
    # Registrar todos los modelos en el registry
    for seed in cfg.SEEDS:
        # MEJORADO: Guardar device en metadata como string
        if 'device' not in metadata:
            metadata['device'] = str(cfg.DEVICE)  # BUG #1 FIX: convertir a string
        
        _model_registry.register_models(
            models_x=seed_models[seed]['models_x'],
            models_y=seed_models[seed]['models_y'],
            scalers=seed_models[seed]['scalers'],
            seed=seed,
            metadata=metadata
        )
        
        # Guardar modelos a disco (backup)
        if cfg.SAVE_MODELS:
            _model_registry.save_to_disk(seed, cfg.MODEL_SAVE_DIR)
    
    # Mantener compatibilidad con variables globales legacy (para inferencia en main)
    global _trained_models, _trained_scalers
    _trained_models = {
        'all_models_x': seed_models[best_seed]['models_x'], 
        'all_models_y': seed_models[best_seed]['models_y']
    }
    _trained_scalers = seed_models[best_seed]['scalers']
    
    # Calcular estadísticas generales de RMSE
    rmse_mean = np.mean(fold_rmse_list)
    rmse_std = np.std(fold_rmse_list)
    
    print("\n" + "="*80)
    print("ESTADÍSTICAS DE RMSE POR FOLD (TODOS)")
    print("="*80)
    for i, rmse in enumerate(fold_rmse_list, 1):
        print(f"Fold {i}: RMSE = {rmse:.5f}")
    print("-"*80)
    print(f"RMSE Promedio Global: {rmse_mean:.5f}")
    print(f"RMSE Desviación Estándar Global: {rmse_std:.5f}")
    print("="*80)

    # [4/4] Inferencia sobre el test usando todos los modelos entrenados + TTA selectivo
    print(f"\n[4/4] Inferencia y submission con ensemble de {len(_trained_models['all_models_x'])} modelos + TTA en mejor semilla...")
    test_seqs, test_meta, feat_cols_t, dir_map_test = prepare_sequences_vectorized(
        test_input, test_template=test_template, is_training=False,
        window_size=cfg.WINDOW_SIZE
    )
    assert feat_cols_t == feat_cols, "¡Las columnas de características de Train/Test no coinciden!"
    
    # PROBLEMA 3 FIX: Liberar test_input inmediatamente después de crear secuencias
    del test_input
    gc.collect()
    print("✓ test_input liberado de memoria")

    idx_x = feat_cols.index('x')
    idx_y = feat_cols.index('y')

    # PROBLEMA 1 FIX: Evitar duplicación - test_seqs ya es lista
    X_test_raw = test_seqs
    x_last_uni = np.array([s[-1, idx_x] for s in X_test_raw], dtype=np.float32)
    y_last_uni = np.array([s[-1, idx_y] for s in X_test_raw], dtype=np.float32)

    # PROBLEMA 4 FIX: Batch processing para arrays grandes
    # Calcular BATCH_SIZE dinámico basado en RAM disponible
    # Objetivo: mantener batch arrays bajo 500 MB
    seq_size = X_test_raw[0].shape[0] * len(feat_cols) * 4  # bytes por secuencia
    TARGET_BATCH_MB = 500  # Mantener batches bajo 500 MB
    BATCH_SIZE = max(100, min(TARGET_BATCH_MB * 1024 * 1024 // seq_size, 5000))  # Entre 100 y 5000
    N = len(X_test_raw)
    H = cfg.MAX_FUTURE_HORIZON
    
    print(f"\n--- Generando predicciones normales de {len(_trained_models['all_models_x'])} modelos (streaming + batches)...")
    print(f"    Processing in batches of {BATCH_SIZE} sequences...")
    
    num_models = len(_trained_models['all_models_x'])
    all_predictions_dx = []
    all_predictions_dy = []
    
    # Procesar test en batches
    for batch_start in range(0, N, BATCH_SIZE):
        batch_end = min(batch_start + BATCH_SIZE, N)
        X_batch = X_test_raw[batch_start:batch_end]
        
        print(f"    Procesando batch {batch_start}-{batch_end} / {N}...")
        
        # Inicializar arrays para este batch
        batch_dx = np.zeros((len(X_batch), H), dtype=np.float32)
        batch_dy = np.zeros((len(X_batch), H), dtype=np.float32)
        
        if cfg.USE_UNCERTAINTY:
            batch_dx_var = np.zeros((len(X_batch), H), dtype=np.float32)
            batch_dy_var = np.zeros((len(X_batch), H), dtype=np.float32)
        
        # Generar predicciones para este batch
        for i, preds in enumerate(generate_predictions_streaming(
            _trained_models['all_models_x'], 
            _trained_models['all_models_y'], 
            _trained_scalers, 
            X_batch, 
            cfg.DEVICE,
            use_uncertainty=cfg.USE_UNCERTAINTY
        )):
            if cfg.USE_UNCERTAINTY:
                pred_x_mean, pred_x_logvar, pred_y_mean, pred_y_logvar = preds
                batch_dx += pred_x_mean / num_models
                batch_dy += pred_y_mean / num_models
                batch_dx_var += np.exp(pred_x_logvar) / num_models
                batch_dy_var += np.exp(pred_y_logvar) / num_models
            else:
                pred_x, pred_y = preds
                batch_dx += pred_x / num_models
                batch_dy += pred_y / num_models
        
        all_predictions_dx.append(batch_dx)
        all_predictions_dy.append(batch_dy)
        
        # Liberar memoria del batch
        del X_batch, batch_dx, batch_dy
        gc.collect()
    
    # Concatenar todas las predicciones
    ens_dx = np.vstack(all_predictions_dx)
    ens_dy = np.vstack(all_predictions_dy)
    
    # Liberar memoria temporal
    del all_predictions_dx, all_predictions_dy
    gc.collect()
    
    print(f"✓ Streaming ensemble completado: {num_models} modelos procesados")
    
    if cfg.USE_UNCERTAINTY:
        print(f"\n--- Estadísticas de Incertidumbre:")
        all_batch_dx_var = []
        all_batch_dy_var = []
        # Recopilar varianzas si existen
        for batch_start in range(0, N, BATCH_SIZE):
            if 'batch_dx_var' in locals():
                all_batch_dx_var.append(batch_dx_var)
                all_batch_dy_var.append(batch_dy_var)
        
        if all_batch_dx_var:
            dx_var = np.vstack(all_batch_dx_var)
            dy_var = np.vstack(all_batch_dy_var)
            print(f"    Varianza X promedio: {dx_var.mean():.4f}")
            print(f"    Varianza Y promedio: {dy_var.mean():.4f}")
            print(f"    Varianza X máxima: {dx_var.max():.4f}")
            print(f"    Varianza Y máxima: {dy_var.max():.4f}")
            del dx_var, dy_var
    
    # Liberar memoria de modelos después del streaming
    del _trained_models['all_models_x'], _trained_models['all_models_y'], _trained_scalers
    gc.collect()

    # TTA solo para la mejor semilla usando streaming (OPTIMIZADO para memoria)
    tta_weight = 0.35  # FIX #3: Aumentado de 0.15 para dar más peso a augmentaciones
    tta_augmentations = [
        ('noise', 0.01),      # Ruido gaussiano pequeño
        ('temporal_shift', 1), # Desplazamiento temporal de 1 frame
        ('speed_scale', 1.05),  # Escala de velocidad 5%
        # Augmentaciones de caos normales
        ('chaos_pressure', None),    # Simular presión
        ('chaos_scrambling', None),  # Simular scrambling
        ('chaos_direction', None),   # Simular cambios de dirección
        # Augmentaciones extremas (nuevas)
        ('chaos_extreme', None),     # Movimientos MUY erráticos
        ('pressure_burst', None),    # Aceleraciones súbitas bajo presión
        ('lateral_escape', None)     # Escape lateral intenso
    ]
    
    print(f"\n--- Aplicando TTA a modelos de la mejor semilla ({best_seed}) con streaming...")
    print(f"    Peso de cada aumentación TTA: {tta_weight:.3f}")
    print(f"    Aumentaciones: {len(tta_augmentations)}")
    
    best_models_x = seed_models[best_seed]['models_x']
    best_models_y = seed_models[best_seed]['models_y']
    best_scalers = seed_models[best_seed]['scalers']
    
    # PROBLEMA 4 FIX: TTA también con batch processing
    print(f"    Initializing TTA arrays with batch processing...")
    tta_dx = np.zeros((N, H), dtype=np.float32)
    tta_dy = np.zeros((N, H), dtype=np.float32)
    tta_count = 0
    
    for aug_name, aug_param in tta_augmentations:
        print(f"    Procesando aumentación: {aug_name} (param={aug_param})...")
        
        all_tta_batch_dx = []
        all_tta_batch_dy = []
        
        # Procesar TTA en batches
        for batch_start in range(0, N, BATCH_SIZE):
            batch_end = min(batch_start + BATCH_SIZE, N)
            X_batch = X_test_raw[batch_start:batch_end]
            
            # ISSUE #1 FIX: Materializar generador para evitar NameError
            if aug_name.startswith('chaos_'):
                chaos_type = aug_name.replace('chaos_', '')
                # Usar función correcta según el tipo
                if chaos_type in ['extreme', 'pressure_burst', 'lateral_escape']:
                    X_test_aug_list = [apply_extreme_chaos_augmentations(seq, feat_cols, aug_name) for seq in X_batch]
                else:
                    X_test_aug_list = [apply_chaos_augmentations(seq, feat_cols, chaos_type) for seq in X_batch]
            else:
                X_test_aug_list = list(apply_tta_streaming(X_batch, aug_name, aug_param, feat_cols))
            
            # Inicializar arrays para este batch de TTA
            batch_tta_dx = np.zeros((len(X_batch), H), dtype=np.float32)
            batch_tta_dy = np.zeros((len(X_batch), H), dtype=np.float32)
            
            # Generar predicciones TTA para este batch
            model_count = 0
            for preds in generate_predictions_streaming(best_models_x, best_models_y, best_scalers, X_test_aug_list, cfg.DEVICE, use_uncertainty=cfg.USE_UNCERTAINTY):
                if cfg.USE_UNCERTAINTY:
                    pred_x_mean, _, pred_y_mean, _ = preds  # Ignorar logvar en TTA
                    batch_tta_dx += pred_x_mean
                    batch_tta_dy += pred_y_mean
                else:
                    pred_x, pred_y = preds
                    batch_tta_dx += pred_x
                    batch_tta_dy += pred_y
                model_count += 1
            
            # Promediar predicciones de modelos para este batch
            if model_count > 0:
                batch_tta_dx /= model_count
                batch_tta_dy /= model_count
            
            all_tta_batch_dx.append(batch_tta_dx)
            all_tta_batch_dy.append(batch_tta_dy)
            
            # Liberar memoria del batch de TTA
            del X_batch, X_test_aug_list, batch_tta_dx, batch_tta_dy
            gc.collect()
        
        # Concatenar predicciones TTA de este batch
        tta_batch_dx = np.vstack(all_tta_batch_dx)
        tta_batch_dy = np.vstack(all_tta_batch_dy)
        
        # Acumular en arrays globales
        tta_dx += tta_batch_dx
        tta_dy += tta_batch_dy
        tta_count += 1
        
        # Liberar memoria de batch
        del all_tta_batch_dx, all_tta_batch_dy, tta_batch_dx, tta_batch_dy
        gc.collect()
    
    # Promediar predicciones TTA
    if tta_count > 0:
        tta_dx /= tta_count
        tta_dy /= tta_count
    
    # Guardar valores para prints finales ANTES de eliminar
    num_best_models = len(best_models_x)
    num_base_models = num_models

    # Ensemble ponderado: predicciones normales (peso 1.0) + TTA (peso reducido)
    print(f"\n--- Combinando predicciones...")
    print(f"    Predicciones normales: {num_models} modelos × peso 1.0")
    print(f"    Predicciones TTA: {tta_count} aumentaciones × peso {tta_weight:.3f}")
    
    # Calcular suma ponderada
    total_weight = num_models * 1.0 + tta_count * tta_weight
    
    # Aplicar pesos
    ens_dx = ens_dx * 1.0 + tta_dx * tta_weight
    ens_dy = ens_dy * 1.0 + tta_dy * tta_weight
    
    # Normalizar por peso total
    ens_dx /= total_weight
    ens_dy /= total_weight
    
    # Liberar memoria de modelos TTA (una sola vez, después del último uso)
    del best_models_x, best_models_y, best_scalers
    gc.collect()
    
    # Liberar memoria de TTA
    del tta_dx, tta_dy
    gc.collect()
    
    # Issue #3: Restricciones físicas post-ensemble (opcional para mantener continuidad)
    if cfg.APPLY_PHYSICS_CONSTRAINTS:
        print(f"\n--- Aplicando restricciones físicas...")
        
        # 1. Velocidad máxima por frame (NFL jugadores ~10 m/s = ~1.0 yards/frame a 10 fps)
        MAX_SPEED_PER_FRAME = 1.0
        violations_count = 0
        
        for i in range(len(ens_dx)):
            for t in range(ens_dx.shape[1] - 1):
                # Calcular desplazamiento entre frames consecutivos
                dx_step = ens_dx[i, t+1] - ens_dx[i, t]
                dy_step = ens_dy[i, t+1] - ens_dy[i, t]
                displacement = np.sqrt(dx_step**2 + dy_step**2)
                
                # Si excede velocidad máxima, escalar al límite
                if displacement > MAX_SPEED_PER_FRAME:
                    scale = MAX_SPEED_PER_FRAME / displacement
                    ens_dx[i, t+1] = ens_dx[i, t] + dx_step * scale
                    ens_dy[i, t+1] = ens_dy[i, t] + dy_step * scale
                    violations_count += 1
        
        print(f"    ✓ Velocidad máxima aplicada: {MAX_SPEED_PER_FRAME:.2f} yards/frame")
        print(f"    ✓ Correcciones realizadas: {violations_count} violaciones")
        
        # 2. Suavizado de trayectoria (Savitzky-Golay)
        try:
            from scipy.signal import savgol_filter
            
            smoothed_count = 0
            for i in range(len(ens_dx)):
                if ens_dx.shape[1] >= 5:  # Mínimo 5 puntos para suavizado
                    ens_dx[i] = savgol_filter(ens_dx[i], window_length=5, polyorder=2)
                    ens_dy[i] = savgol_filter(ens_dy[i], window_length=5, polyorder=2)
                    smoothed_count += 1
            
            print(f"    ✓ Suavizado Savitzky-Golay aplicado: {smoothed_count} trayectorias")
        except ImportError:
            print(f"    ⚠ scipy.signal no disponible, omitiendo suavizado")
        
        print(f"    ✓ Restricciones físicas completadas")
    else:
        print(f"\n--- Restricciones físicas desactivadas (cfg.APPLY_PHYSICS_CONSTRAINTS=False)")
    
    H = ens_dx.shape[1]

    # Construcción de las filas para la submission, con inversión para jugadas a la derecha
    rows = []
    # Usar operaciones nativas de Polars
    tt_sorted = test_template.sort(['game_id', 'play_id', 'nfl_id'])
    
    # Crear un diccionario de acceso rápido para frame_id
    frame_id_map = {}
    for row in tt_sorted.iter_rows(named=True):
        key = (row['game_id'], row['play_id'], row['nfl_id'])
        if key not in frame_id_map:
            frame_id_map[key] = []
        frame_id_map[key].append(row['frame_id'])

    for i, meta in enumerate(test_meta):
        gid = meta['game_id']; pid = meta['play_id']; nid = meta['nfl_id']
        play_dir = meta['play_direction']
        play_is_right = (play_dir == 'right')

        try:
            fids = frame_id_map[(gid, pid, nid)]
            fids = sorted(fids)
        except KeyError:
            continue

        for t, fid in enumerate(fids):
            tt = min(t, H - 1)
            x_uni, y_uni = clip_to_field(x_last_uni[i] + ens_dx[i, tt], y_last_uni[i] + ens_dy[i, tt])
            x_out, y_out = invert_to_original_direction(x_uni, y_uni, play_is_right)

            rows.append({
                'id': f"{gid}_{pid}_{nid}_{int(fid)}",
                'x': x_out,
                'y': y_out
            })

    # NO guardar submission.parquet aquí - el servidor lo hace automáticamente
    print("\n" + "="*80)
    print("¡PASO 2 COMPLETO CON TTA SELECTIVO Y RESTRICCIONES FÍSICAS!")
    print("="*80)
    print(f"✓ {len(rows)} predicciones generadas")
    print(f"Total de modelos base en ensemble: {num_base_models}")
    print(f"Modelos con TTA (semilla {best_seed}): {num_best_models} × {len(tta_augmentations)} aumentaciones")
    print(f"Peso efectivo TTA vs normal: {tta_weight:.3f} : 1.0")
    print(f"Características utilizadas: {len(feat_cols)}  (Polars activo: True)")
    print("\n✓ Proceso completado. El servidor generará submission.parquet automáticamente.")

# ============================================================================
# KAGGLE EVALUATION SETUP
# ============================================================================

if __name__ == "__main__":
    # Crear inference_server dentro de __main__ para evitar errores al importar
    inference_server = kaggle_evaluation.nfl_inference_server.NFLInferenceServer(predict)
    
    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        # Modo 1: Evaluación Kaggle (hidden test set)
        # inference_server.serve() debe llamarse en < 10 minutos
        # La primera llamada a predict() puede tardar horas (entrenamiento)
        inference_server.serve()
    else:
        # Modo 2: Local gateway (mismo flujo que Kaggle)
        # Entrena en primera llamada a predict(), genera submission automáticamente
        print("Modo local: usando gateway para consistencia con Kaggle...")
        print("Entrenamiento ocurrirá en la primera llamada a predict()")
        try:
            inference_server.run_local_gateway(
                ('/kaggle/input/nfl-big-data-bowl-2026-prediction/',)
            )
        except KeyboardInterrupt:
            print("\n⚠ Proceso interrumpido por el usuario")
        except Exception as e:
            print(f"\n❌ Error durante la ejecución: {e}")
            import traceback
            traceback.print_exc()

Modo local: usando gateway para consistencia con Kaggle...
Entrenamiento ocurrirá en la primera llamada a predict()
Predicting for 33 samples...
ENTRENAMIENTO INICIAL (primera llamada a predict)
Esto puede tardar varias horas - sin límite de tiempo
✓ ModelRegistry cleared
PASO 2: PIPELINE MEJORADO CON ENSEMBLE DE 1 SEMILLAS + TTA SELECTIVO
Semillas a utilizar: [14]
Polars backend activo: True

[1/4] Cargando datos...
✓ Optimizando tipos de datos para memoria...
✓ Convertidas 8 + 2 + 8 columnas a float32

[2/4] Construyendo secuencias con características AVANZADAS...

PREPARING SEQUENCES WITH VECTORIZED POLARS JOINS
Window size: 10
play_direction merge OK: {'right': 282609, 'left': 280327}
Step 1/3: Adding basic features...

Step 2/3: Adding selected advanced features...
  [+] Added 'distance_rate         ' ( 3 cols) in  1056.2 ms
  [+] Added 'target_alignment      ' ( 3 cols) in   167.3 ms
  [+] Added 'multi_window_rolling  ' (24 cols) in 125765.2 ms
  [+] Added 'extended_lags         

In [3]:
# import torch
# import torch.nn as nn
# import numpy as np
# import pandas as pd
# from pathlib import Path
# from tqdm.auto import tqdm
# import warnings
# import os

# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import GroupKFold
# from torch.utils.data import TensorDataset, DataLoader

# warnings.filterwarnings('ignore')

# # ---------------------- GPU DETECTION (multi-GPU) ---------------------- #
# def _detect_gpu_devices() -> tuple[bool, list[int] | None]:
#     """
#     Detecta GPUs disponibles para PyTorch.
#     Retorna: (USE_GPU, DEVICE_IDS)
#     """
#     if not torch.cuda.is_available():
#         return False, None
    
#     n_gpus = torch.cuda.device_count()
#     if n_gpus == 0:
#         return False, None
#     elif n_gpus >= 2:
#         device_ids = list(range(n_gpus))
#         return True, device_ids
#     else:
#         return True, [0]

# USE_GPU, DEVICE_IDS = _detect_gpu_devices()

# print("="*80)
# print("GPU CONFIGURATION")
# print("="*80)
# if USE_GPU:
#     print(f"✓ CUDA available: {torch.cuda.is_available()}")
#     print(f"✓ GPU count: {torch.cuda.device_count()}")
#     print(f"✓ Device IDs: {DEVICE_IDS}")
#     if DEVICE_IDS and len(DEVICE_IDS) > 0:
        # for device_id in DEVICE_IDS:
#             print(f"  - GPU {device_id}: {torch.cuda.get_device_name(device_id)}")
#     print(f"✓ CUDA_VISIBLE_DEVICES: {os.environ.get('CUDA_VISIBLE_DEVICES', 'Not set')}")
#     print(f"✓ Will use: {'Multi-GPU (DataParallel)' if DEVICE_IDS and len(DEVICE_IDS) > 1 else 'Single GPU'}")
# else:
#     print("⚠ No GPU detected, will use CPU")
# print("="*80 + "\n")

# # Config
# class Config:
#     DATA_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction/")
    
#     SEED = 42
#     N_FOLDS = 5
#     BATCH_SIZE = 512
#     EPOCHS = 120
#     PATIENCE = 20
#     LEARNING_RATE = 5e-4
    
#     WINDOW_SIZE = 14
#     HIDDEN_DIM = 192
#     MAX_FUTURE_HORIZON = 94
    
#     FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
#     FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
    
#     # GPU Configuration
#     DEVICE = torch.device("cuda" if USE_GPU else "cpu")
#     USE_MULTI_GPU = USE_GPU and DEVICE_IDS and len(DEVICE_IDS) > 1
#     DEVICE_IDS = DEVICE_IDS

# def set_seed(seed=42):
#     import random
#     random.seed(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     torch.backends.cudnn.deterministic = True

# set_seed(Config.SEED)


# def height_to_feet(height_str):
#     try:
#         ft, inches = map(int, str(height_str).split('-'))
#         return ft + inches/12
#     except:
#         return 6.0

# def add_advanced_features(df):
#     """Enhanced feature engineering from Notebook 1"""
#     print("Adding advanced features...")
#     df = df.copy()
#     df = df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
#     gcols = ['game_id', 'play_id', 'nfl_id']
    
#     # Distance Rate Features
#     if 'distance_to_ball' in df.columns:
#         df['distance_to_ball_change'] = df.groupby(gcols)['distance_to_ball'].diff().fillna(0)
#         df['distance_to_ball_accel'] = df.groupby(gcols)['distance_to_ball_change'].diff().fillna(0)
#         df['time_to_intercept'] = (df['distance_to_ball'] / 
#                                     (np.abs(df['distance_to_ball_change']) + 0.1)).clip(0, 10)
    
    # # Target Alignment Features
    # if 'ball_direction_x' in df.columns:
    #     df['velocity_alignment'] = (
    #         df['velocity_x'] * df['ball_direction_x'] +
    #         df['velocity_y'] * df['ball_direction_y']
    #     )
    #     df['velocity_perpendicular'] = (
    #         df['velocity_x'] * (-df['ball_direction_y']) +
    #         df['velocity_y'] * df['ball_direction_x']
    #     )
    #     if 'acceleration_x' in df.columns:
    #         df['accel_alignment'] = (
    #             df['acceleration_x'] * df['ball_direction_x'] +
    #             df['acceleration_y'] * df['ball_direction_y']
    #         )
    
    # # Multi-Window Rolling
    # for window in [3, 5, 10]:
    #     for col in ['velocity_x', 'velocity_y', 's', 'a']:
    #         if col in df.columns:
    #             df[f'{col}_roll{window}'] = df.groupby(gcols)[col].transform(
    #                 lambda x: x.rolling(window, min_periods=1).mean()
    #             )
    #             df[f'{col}_std{window}'] = df.groupby(gcols)[col].transform(
    #                 lambda x: x.rolling(window, min_periods=1).std()
    #             ).fillna(0)
    
    # # Extended Lag Features
    # for lag in [4, 5]:
    #     for col in ['x', 'y', 'velocity_x', 'velocity_y']:
    #         if col in df.columns:
    #             df[f'{col}_lag{lag}'] = df.groupby(gcols)[col].shift(lag).fillna(0)
    
    # # Velocity Change Features
    # if 'velocity_x' in df.columns:
    #     df['velocity_x_change'] = df.groupby(gcols)['velocity_x'].diff().fillna(0)
    #     df['velocity_y_change'] = df.groupby(gcols)['velocity_y'].diff().fillna(0)
    #     df['speed_change'] = df.groupby(gcols)['s'].diff().fillna(0)
    #     df['direction_change'] = df.groupby(gcols)['dir'].diff().fillna(0)
    #     df['direction_change'] = df['direction_change'].apply(
    #         lambda x: x if abs(x) < 180 else x - 360 * np.sign(x)
    #     )
    
    # # Field Position Features
    # df['dist_from_left'] = df['y']
    # df['dist_from_right'] = 53.3 - df['y']
    # df['dist_from_sideline'] = np.minimum(df['dist_from_left'], df['dist_from_right'])
    # df['dist_from_endzone'] = np.minimum(df['x'], 120 - df['x'])
    
    # # Role-Specific Features
    # if 'is_receiver' in df.columns and 'velocity_alignment' in df.columns:
    #     df['receiver_optimality'] = df['is_receiver'] * df['velocity_alignment']
    #     df['receiver_deviation'] = df['is_receiver'] * np.abs(df.get('velocity_perpendicular', 0))
    # if 'is_coverage' in df.columns and 'closing_speed' in df.columns:
    #     df['defender_closing_speed'] = df['is_coverage'] * df['closing_speed']
    
    # # Time Features
    # df['frames_elapsed'] = df.groupby(gcols).cumcount()
    # df['normalized_time'] = df.groupby(gcols)['frames_elapsed'].transform(
    #     lambda x: x / (x.max() + 1)
    # )
    
#     return df

# def add_player_interactions(df):
#     """
#     Agrega features de interacciones jugador-jugador
#     """
#     print("Computing player-player interactions...")
#     df = df.sort_values(['game_id', 'play_id', 'frame_id', 'nfl_id'])
    
#     interactions = []
    
#     for (gid, pid, fid), frame_df in tqdm(df.groupby(['game_id', 'play_id', 'frame_id']), desc="Player interactions"):
#         frame_df = frame_df.reset_index(drop=True)
        
#         # Coordenadas de todos los jugadores
#         coords = frame_df[['x', 'y']].values
#         roles = frame_df['player_role'].values
#         sides = frame_df['player_side'].values
#         velocities = frame_df[['velocity_x', 'velocity_y']].values
        
#         n_players = len(frame_df)
        
#         for i in range(n_players):
#             player_x, player_y = coords[i]
#             player_role = roles[i]
#             player_side = sides[i]
#             player_vx, player_vy = velocities[i]
            
#             # Distancias a todos los demás jugadores
#             dists = np.sqrt((coords[:, 0] - player_x)**2 + (coords[:, 1] - player_y)**2)
#             dists[i] = np.inf  # Ignorar distancia a sí mismo
            
#             # Máscaras por lado
            # same_side = sides == player_side
            # # opp_side = sides != player_side
            
            # # ===== INTERACCIONES CON COMPAÑEROS =====
            # same_dists = dists.copy()
            # same_dists[~same_side] = np.inf
            
            # if np.any(same_dists < np.inf):
            #     nearest_same_idx = np.argmin(same_dists)
            #     nearest_same_dist = same_dists[nearest_same_idx]
            #     nearest_same_angle = np.arctan2(
            #         coords[nearest_same_idx, 1] - player_y,
            #         coords[nearest_same_idx, 0] - player_x
            #     )
            #     nearest_same_vx, nearest_same_vy = velocities[nearest_same_idx]
            #     rel_vx_same = nearest_same_vx - player_vx
            #     rel_vy_same = nearest_same_vy - player_vy
            # else:
            #     nearest_same_dist = 999.0
            #     nearest_same_angle = 0.0
            #     rel_vx_same = rel_vy_same = 0.0
            
            # # ===== INTERACCIONES CON OPONENTES =====
            # opp_dists = dists.copy()
            # opp_dists[~opp_side] = np.inf
            
            # if np.any(opp_dists < np.inf):
            #     # 3 oponentes más cercanos
            #     top3_opp = np.argsort(opp_dists)[:3]
                
            #     opp1_dist = opp_dists[top3_opp[0]] if len(top3_opp) > 0 else 999.0
            #     opp1_angle = np.arctan2(coords[top3_opp[0], 1] - player_y, 
            #                             coords[top3_opp[0], 0] - player_x) if len(top3_opp) > 0 else 0.0
            #     opp1_vx, opp1_vy = velocities[top3_opp[0]] if len(top3_opp) > 0 else (0, 0)
            #     rel_vx_opp1 = opp1_vx - player_vx
            #     rel_vy_opp1 = opp1_vy - player_vy
                
            #     opp2_dist = opp_dists[top3_opp[1]] if len(top3_opp) > 1 else 999.0
            #     opp3_dist = opp_dists[top3_opp[2]] if len(top3_opp) > 2 else 999.0
                
            #     # Promedio de los 3 más cercanos
            #     opp_avg_dist = np.mean([d for d in [opp1_dist, opp2_dist, opp3_dist] if d < 999.0])
                
            #     # "Presión defensiva" (densidad de oponentes en radio de 5 yards)
            #     pressure = np.sum(opp_dists < 5.0)
            # else:
            #     opp1_dist = opp2_dist = opp3_dist = opp_avg_dist = 999.0
            #     opp1_angle = 0.0
            #     rel_vx_opp1 = rel_vy_opp1 = 0.0
            #     pressure = 0
            
            # # ===== ESPACIO ABIERTO (Voronoi approximation) =====
            # # Espacio = distancia promedio a los 5 jugadores más cercanos
            # top5_dists = np.sort(dists)[:5]
            # open_space = np.mean(top5_dists[top5_dists < np.inf]) if np.any(top5_dists < np.inf) else 999.0
            
            # interactions.append({
            #     'game_id': gid,
            #     'play_id': pid,
            #     'frame_id': fid,
            #     'nfl_id': frame_df.loc[i, 'nfl_id'],
            #     # Compañeros
            #     'nearest_teammate_dist': nearest_same_dist,
#                 'nearest_teammate_angle': nearest_same_angle,
#                 'rel_vx_teammate': rel_vx_same,
#                 'rel_vy_teammate': rel_vy_same,
#                 # Oponentes
#                 'nearest_opp_dist': opp1_dist,
#                 'nearest_opp_angle': opp1_angle,
#                 'rel_vx_opp': rel_vx_opp1,
#                 'rel_vy_opp': rel_vy_opp1,
#                 'second_nearest_opp_dist': opp2_dist,
#                 'third_nearest_opp_dist': opp3_dist,
#                 'avg_3nearest_opp_dist': opp_avg_dist,
#                 'defensive_pressure': pressure,
#                 # Espacio
#                 'open_space': open_space,
#             })
    
#     interactions_df = pd.DataFrame(interactions)
#     df = df.merge(interactions_df, on=['game_id', 'play_id', 'frame_id', 'nfl_id'], how='left')
    
#     return df


# def add_team_aggregations(df):
#     """
#     Agrega features de agregaciones por equipo/rol
#     """
#     print("Computing team aggregations...")
#     df = df.copy()
    
#     # Agregaciones por (game, play, frame, side)
#     grp_side = df.groupby(['game_id', 'play_id', 'frame_id', 'player_side'], sort=False)
    
#     df['team_centroid_x'] = grp_side['x'].transform('mean')
#     df['team_centroid_y'] = grp_side['y'].transform('mean')
#     df['team_spread_x'] = grp_side['x'].transform('std').fillna(0)
#     df['team_spread_y'] = grp_side['y'].transform('std').fillna(0)
#     df['team_avg_speed'] = grp_side['s'].transform('mean')
#     df['team_avg_accel'] = grp_side['a'].transform('mean')
    
#     # Distancia al centroide del equipo
#     df['dist_to_team_centroid'] = np.sqrt(
#         (df['x'] - df['team_centroid_x'])**2 + 
#         (df['y'] - df['team_centroid_y'])**2
#     )
    
#     # Agregaciones por rol
#     grp_role = df.groupby(['game_id', 'play_id', 'frame_id', 'player_role'], sort=False)
#     df['role_avg_x'] = grp_role['x'].transform('mean')
#     df['role_avg_y'] = grp_role['y'].transform('mean')
#     df['role_avg_speed'] = grp_role['s'].transform('mean')
#     df['role_count'] = grp_role['nfl_id'].transform('count')
    
#     # Velocidad del equipo hacia el balón
#     df['team_velocity_to_ball_x'] = grp_side['velocity_x'].transform('mean')
#     df['team_velocity_to_ball_y'] = grp_side['velocity_y'].transform('mean')
    
#     return df


# def add_field_zone_features(df):
#     """
#     Agrega features de zona del campo
#     """
#     print("Computing field zone features...")
#     df = df.copy()
    
#     # Zona horizontal (red zone, midfield, etc)
#     df['in_red_zone'] = ((df['x'] >= 90) | (df['x'] <= 30)).astype(int)
#     df['in_midfield'] = ((df['x'] >= 40) & (df['x'] <= 80)).astype(int)
    
    # Distancia a endzones
    # df['dist_to_nearest_endzone'] = np.minimum(df['x'], 120 - df['x'])
    # df['dist_to_offensive_endzone'] = 120 - df['x']  # Asumiendo offense va hacia x=120
    
    # Distancia a sidelines (ya tienes dist_from_sideline)
    # Añadir cuadrante del campo
#     df['field_quadrant_x'] = pd.cut(df['x'], bins=[0, 30, 60, 90, 120], labels=[0, 1, 2, 3]).astype(float)
#     df['field_quadrant_y'] = pd.cut(df['y'], bins=[0, 13.325, 26.65, 39.975, 53.3], labels=[0, 1, 2, 3]).astype(float)
    
#     # Ángulo relativo a orientación del campo (hacia endzone)
#     field_direction_rad = np.radians(0)  # Campo va en dirección 0° (hacia +x)
#     df['angle_to_endzone'] = np.abs(np.radians(df['dir']) - field_direction_rad)
    
#     return df


# def add_motion_derivatives(df):
#     """
#     Agrega derivadas de movimiento: jerk, curvatura, eficiencia
#     """
#     print("Computing motion derivatives...")
#     df = df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id']).copy()
#     gcols = ['game_id', 'play_id', 'nfl_id']
    
#     # Jerk (cambio de aceleración)
#     df['jerk_x'] = df.groupby(gcols)['acceleration_x'].diff().fillna(0)
#     df['jerk_y'] = df.groupby(gcols)['acceleration_y'].diff().fillna(0)
#     df['jerk_magnitude'] = np.sqrt(df['jerk_x']**2 + df['jerk_y']**2)
    
#     # Curvatura (cambio de dirección)
#     df['direction_change'] = df.groupby(gcols)['dir'].diff().fillna(0)
#     df['direction_change'] = np.abs(df['direction_change'])
#     df['direction_change'] = np.where(df['direction_change'] > 180, 360 - df['direction_change'], df['direction_change'])
    
#     # Distancia euclidiana vs distancia recorrida (eficiencia)
#     df['cumulative_dist'] = df.groupby(gcols).apply(
#         lambda g: np.sqrt(g['velocity_x'].diff()**2 + g['velocity_y'].diff()**2).cumsum()
#     ).reset_index(level=[0, 1, 2], drop=True)
    
#     first_x = df.groupby(gcols)['x'].transform('first')
#     first_y = df.groupby(gcols)['y'].transform('first')
#     df['euclidean_dist'] = np.sqrt((df['x'] - first_x)**2 + (df['y'] - first_y)**2)
#     df['movement_efficiency'] = df['euclidean_dist'] / (df['cumulative_dist'] + 1e-6)
    
#     # Predicción lineal simple (baseline extrapolation)
#     df['predicted_x_linear'] = df['x'] + df['velocity_x'] * 1.0  # 1 frame adelante
#     df['predicted_y_linear'] = df['y'] + df['velocity_y'] * 1.0
    
#     return df


# def add_play_context_features(df):
#     """
#     Agrega contexto fijo de la jugada usando ball_land
#     """
#     print("Computing play context features...")
#     df = df.copy()
    
#     # Distancia inicial al destino del balón (contexto fijo por jugada)
#     first_frame = df.groupby(['game_id', 'play_id', 'nfl_id']).first().reset_index()
#     first_frame['initial_dist_to_ball_land'] = np.sqrt(
#         (first_frame['ball_land_x'] - first_frame['x'])**2 + 
#         (first_frame['ball_land_y'] - first_frame['y'])**2
#     )
#     first_frame['initial_angle_to_ball_land'] = np.arctan2(
#         first_frame['ball_land_y'] - first_frame['y'],
#         first_frame['ball_land_x'] - first_frame['x']
#     )
    
#     # Tipo de ruta inferida
#     first_frame['pass_distance'] = np.abs(first_frame['ball_land_x'] - first_frame['x'])
#     first_frame['pass_type'] = pd.cut(
#         first_frame['pass_distance'], 
#         bins=[0, 10, 20, 999], 
#         labels=['short', 'medium', 'deep']
#     ).astype(str)
#     first_frame['pass_type_short'] = (first_frame['pass_type'] == 'short').astype(int)
#     first_frame['pass_type_medium'] = (first_frame['pass_type'] == 'medium').astype(int)
#     first_frame['pass_type_deep'] = (first_frame['pass_type'] == 'deep').astype(int)
    
#     # Merge back
#     context_cols = ['game_id', 'play_id', 'nfl_id', 'initial_dist_to_ball_land', 
#                     'initial_angle_to_ball_land', 'pass_type_short', 'pass_type_medium', 'pass_type_deep']
#     df = df.merge(first_frame[context_cols], on=['game_id', 'play_id', 'nfl_id'], how='left')
    
#     return df


# def prepare_combined_features(input_df, output_df=None, test_template=None, is_training=True, window_size=10):
#     """COMBINED: Advanced features + enhanced preprocessing + EXPERT FEATURES"""
#     print(f"Preparing EXPERT sequences (window_size={window_size})...")
    
#     input_df = input_df.copy()
    
    # # BASIC FEATURES (existing)
    # input_df['player_height_feet'] = input_df['player_height'].apply(height_to_feet)
    
    # dir_rad = np.deg2rad(input_df['dir'].fillna(0))
    # o_rad = np.deg2rad(input_df['o'].fillna(0))
    
    # input_df['velocity_x'] = input_df['s'] * np.sin(dir_rad)
    # input_df['velocity_y'] = input_df['s'] * np.cos(dir_rad)
    # input_df['acceleration_x'] = input_df['a'] * np.sin(dir_rad)
    # input_df['acceleration_y'] = input_df['a'] * np.cos(dir_rad)
    # input_df['orientation_x'] = np.sin(o_rad)
    # input_df['orientation_y'] = np.cos(o_rad)
    
    # input_df['is_offense'] = (input_df['player_side'] == 'Offense').astype(int)
    # input_df['is_defense'] = (input_df['player_side'] == 'Defense').astype(int)
    # input_df['is_receiver'] = (input_df['player_role'] == 'Targeted Receiver').astype(int)
    # input_df['is_coverage'] = (input_df['player_role'] == 'Defensive Coverage').astype(int)
    # input_df['is_passer'] = (input_df['player_role'] == 'Passer').astype(int)
    # input_df['is_rusher'] = (input_df['player_role'] == 'Pass Rusher').astype(int)
    
    # input_df['field_x_norm'] = (input_df['x'] - Config.FIELD_X_MIN) / (Config.FIELD_X_MAX - Config.FIELD_X_MIN)
    # input_df['field_y_norm'] = (input_df['y'] - Config.FIELD_Y_MIN) / (Config.FIELD_Y_MAX - Config.FIELD_Y_MIN)
    
    # mass_kg = input_df['player_weight'].fillna(200.0) / 2.20462
    # input_df['momentum_x'] = input_df['velocity_x'] * mass_kg
    # input_df['momentum_y'] = input_df['velocity_y'] * mass_kg
    # input_df['kinetic_energy'] = 0.5 * mass_kg * (input_df['s'] ** 2)
    
    # if 'ball_land_x' in input_df.columns:
    #     dx_ball = input_df['ball_land_x'] - input_df['x']
    #     dy_ball = input_df['ball_land_y'] - input_df['y']
    #     input_df['distance_to_ball'] = np.sqrt(dx_ball**2 + dy_ball**2)
    #     input_df['angle_to_ball'] = np.arctan2(dy_ball, dx_ball)
    #     input_df['ball_direction_x'] = dx_ball / (input_df['distance_to_ball'] + 1e-6)
    #     input_df['ball_direction_y'] = dy_ball / (input_df['distance_to_ball'] + 1e-6)
    #     input_df['velocity_alignment'] = (
    #         input_df['velocity_x'] * input_df['ball_direction_x'] + 
    #         input_df['velocity_y'] * input_df['ball_direction_y']
    #     )
    
    # # Sort for temporal features
    # input_df = input_df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    # gcols = ['game_id', 'play_id', 'nfl_id']
    
    # # Temporal features (existing)
    # for lag in [1, 2, 3, 5]:
    #     for col in ['x', 'y', 's', 'a', 'dir']:
    #         input_df[f'{col}_lag{lag}'] = input_df.groupby(gcols)[col].shift(lag).fillna(0)
    
    # for alpha in [0.1, 0.3, 0.5]:
    #     for col in ['s', 'a', 'velocity_x', 'velocity_y']:
    #         input_df[f'{col}_ema{int(alpha*10)}'] = input_df.groupby(gcols)[col].transform(
    #             lambda x: x.ewm(alpha=alpha, adjust=False).mean()
    #         )
    
    # # ADVANCED FEATURES (existing)
    # input_df = add_advanced_features(input_df)
    
    # # ===== NEW EXPERT FEATURES =====
    # input_df = add_player_interactions(input_df)
    # input_df = add_team_aggregations(input_df)
    # input_df = add_field_zone_features(input_df)
    # input_df = add_motion_derivatives(input_df)
    # input_df = add_play_context_features(input_df)
    
    # # Fill NaNs
    # input_df = input_df.fillna(0)
    
    # # CREATE SEQUENCES 
 
    # input_df.set_index(['game_id', 'play_id', 'nfl_id'], inplace=True)
    # grouped = input_df.groupby(level=['game_id', 'play_id', 'nfl_id'])
    
    # target_rows = output_df if is_training else test_template
    # target_groups = target_rows[['game_id', 'play_id', 'nfl_id']].drop_duplicates();
    
    # sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids = [], [], [], [], []
    
    # for _, row in tqdm(target_groups.iterrows(), total=len(target_groups)):
    #     key = (row['game_id'], row['play_id'], row['nfl_id'])
        
    #     try:
    #         group_df = grouped.get_group(key)
    #     except KeyError:
    #         continue
        
    #     input_window = group_df.tail(window_size)
        
    #     if len(input_window) < window_size:
    #         if is_training:
    #             continue
    #         pad_len = window_size - len(input_window)
    #         pad_df = pd.DataFrame(np.nan, index=range(pad_len), columns=input_window.columns)
    #         input_window = pd.concat([pad_df, input_window], ignore_index=True)
        
    #     # Enhanced imputation
        # input_window = input_window.fillna(method='ffill').fillna(method='bfill')
        # input_window = input_window.fillna(group_df.mean(numeric_only=True))
        
        # seq = input_window[feature_cols].values
        
        # if np.isnan(seq).any():
        #     if is_training:
        #         continue
        #     seq = np.nan_to_num(seq, nan=0.0)
        
        # sequences.append(seq)
        
        # if is_training:
        #     out_grp = output_df[
        #         (output_df['game_id']==row['game_id']) &
        #         (output_df['play_id']==row['play_id']) &
        #         (output_df['nfl_id']==row['nfl_id'])
        #     ].sort_values('frame_id')
            
        #     last_x = input_window.iloc[-1]['x']
        #     last_y = input_window.iloc[-1]['y']
            
        #     dx = out_grp['x'].values - last_x
        #     dy = out_grp['y'].values - last_y
            
        #     targets_dx.append(dx)
        #     targets_dy.append(dy)
        #     targets_frame_ids.append(out_grp['frame_id'].values)
        
        # sequence_ids.append({
        #     'game_id': key[0],
        #     'play_id': key[1],
        #     'nfl_id': key[2],
        #     'frame_id': input_window.iloc[-1]['frame_id']
#         })
    
#     print(f"Created {len(sequences)} sequences")
    
#     if is_training:
#         return sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids, feature_cols
#     return sequences, sequence_ids, feature_cols


# class EnhancedSeqModel(nn.Module):
#     def __init__(self, input_dim, horizon):
#         super().__init__()
#         self.horizon = horizon
        
#         self.gru = nn.GRU(input_dim, 192, num_layers=3, batch_first=True, dropout=0.2, bidirectional=False)
        
#         self.conv1d = nn.Sequential(
#             nn.Conv1d(192, 128, kernel_size=3, padding=1),
#             nn.GELU(),
#             nn.Conv1d(128, 128, kernel_size=5, padding=2),
#             nn.GELU(),
#         )
        
#         self.pool_ln = nn.LayerNorm(192)
#         self.pool_attn = nn.MultiheadAttention(192, num_heads=8, batch_first=True, dropout=0.1)
#         self.pool_query = nn.Parameter(torch.randn(1, 1, 192))
        
#         self.head = nn.Sequential(
#             nn.Linear(192 + 128, 256),
#             nn.GELU(),
#             nn.Dropout(0.3),
#             nn.Linear(256, 128),
#             nn.GELU(),
#             nn.Dropout(0.2),
#             nn.Linear(128, horizon * 2)
    #     )
        
    #     self.initialize_weights()
    
    # def initialize_weights(self):
    #     for module in self.modules():
    #         if isinstance(module, nn.Linear):
    #             nn.init.xavier_uniform_(module.weight)
    #             if module.bias is not None:
    #                 nn.init.constant_(module.bias, 0)
    #         elif isinstance(module, nn.GRU):
    #             for name, param in module.named_parameters():
    #                 if 'weight' in name:
    #                     nn.init.orthogonal_(param)
    #                 elif 'bias' in name:
    #                     nn.init.constant_(param, 0)
    
    # def forward(self, x):
    #     h, _ = self.gru(x)
        
    #     h_conv = self.conv1d(h.transpose(1, 2)).transpose(1, 2)
    #     h_conv_pool = h_conv.mean(dim=1)
        
    #     B = h.size(0)
    #     q = self.pool_query.expand(B, -1, -1)
    #     h_norm = self.pool_ln(h)
    #     ctx, _ = self.pool_attn(q, h_norm, h_norm)
    #     ctx = ctx.squeeze(1)
        
    #     combined = torch.cat([ctx, h_conv_pool], dim=1)
        
    #     out = self.head(combined)
    #     out = out.view(B, 2, self.horizon)
        
#         out = torch.cumsum(out, dim=2)
        
#         return out[:, 0, :], out[:, 1, :]


# class EnhancedTemporalLoss(nn.Module):
#     def __init__(self, delta=0.5, time_decay=0.05, velocity_weight=0.1):
#         super().__init__()
#         self.delta = delta
#         self.time_decay = time_decay
#         self.velocity_weight = velocity_weight
#         self.huber = nn.SmoothL1Loss(reduction='none')
    
#     def forward(self, pred_dx, pred_dy, target_dx, target_dy, mask):
#         L = pred_dx.size(1)
#         t = torch.arange(L, device=pred_dx.device).float()
#         time_weights = torch.exp(-self.time_decay * t).view(1, L)
        
#         loss_dx = self.huber(pred_dx, target_dx) * time_weights
#         loss_dy = self.huber(pred_dy, target_dy) * time_weights
        
#         masked_loss_dx = (loss_dx * mask).sum() / (mask.sum() + 1e-8)
#         masked_loss_dy = (loss_dy * mask).sum() / (mask.sum() + 1e-8)
        
#         position_loss = (masked_loss_dx + masked_loss_dy) / 2
        
#         if self.velocity_weight > 0:
#             pred_velocity_x = torch.diff(pred_dx, dim=1, prepend=torch.zeros_like(pred_dx[:, :1]))
#             pred_velocity_y = torch.diff(pred_dy, dim=1, prepend=torch.zeros_like(pred_dy[:, :1]))
#             target_velocity_x = torch.diff(target_dx, dim=1, prepend=torch.zeros_like(target_dx[:, :1]))
#             target_velocity_y = torch.diff(target_dy, dim=1, prepend=torch.zeros_like(target_dy[:, :1]))
            
#             velocity_loss = (
#                 self.huber(pred_velocity_x, target_velocity_x).mean() +
#                 self.huber(pred_velocity_y, target_velocity_y).mean()
#             ) * self.velocity_weight
            
#             total_loss = position_loss + velocity_loss
#         else:
#             total_loss = position_loss
        
#         return total_loss

# def compute_rmse(pred_dx, pred_dy, target_dx, target_dy, mask):
#     """Calculate RMSE for position predictions WITH 0.5 factor"""
#     squared_errors_x = ((pred_dx - target_dx)**2) * mask
#     squared_errors_y = ((pred_dy - target_dy)**2) * mask
    
#     mse_x = squared_errors_x.sum() / (mask.sum() + 1e-8)
#     mse_y = squared_errors_y.sum() / (mask.sum() + 1e-8)
    
#     # Apply the 0.5 factor as in competition scoring
#     combined_mse = 0.5 * (mse_x + mse_y)
#     return torch.sqrt(combined_mse).item()

# # Also update the OOF RMSE calculation:
# def calculate_oof_rmse(sequences, targets_dx, targets_dy, oof_predictions):
#     """Calculate overall OOF RMSE with 0.5 factor"""
#     all_squared_errors_x, all_squared_errors_y = [], []
#     all_weights = []
    
#     for i in range(len(sequences)):
#         target_dx = targets_dx[i]
#         target_dy = targets_dy[i]
#         pred_dx = oof_predictions[i, :len(target_dx), 0]
#         pred_dy = oof_predictions[i, :len(target_dy), 1]
        
#         squared_errors_x = (pred_dx - target_dx)**2
#         squared_errors_y = (pred_dy - target_dy)**2
        
#         all_squared_errors_x.extend(squared_errors_x)
#         all_squared_errors_y.extend(squared_errors_y)
#         all_weights.extend([1.0] * len(target_dx))
    
#     # Weighted average with 0.5 factor
#     mse_x = np.average(all_squared_errors_x, weights=all_weights)
#     mse_y = np.average(all_squared_errors_y, weights=all_weights)
#     oof_rmse = np.sqrt(0.5 * (mse_x + mse_y))
    
#     return oof_rmse

# def prepare_targets_enhanced(batch_dx, batch_dy, max_h):
#     tensors_dx, tensors_dy, masks = [], [], []
#     for dx_arr, dy_arr in zip(batch_dx, batch_dy):
#         L = len(dx_arr)
#         padded_dx = np.pad(dx_arr, (0, max_h - L), constant_values=0).astype(np.float32)
#         padded_dy = np.pad(dy_arr, (0, max_h - L), constant_values=0).astype(np.float32)
#         mask = np.zeros(max_h, dtype=np.float32)
#         mask[:L] = 1.0
#         tensors_dx.append(torch.tensor(padded_dx))
#         tensors_dy.append(torch.tensor(padded_dy))
#         masks.append(torch.tensor(mask))
#     return torch.stack(tensors_dx), torch.stack(tensors_dy), torch.stack(masks)

# def train_model_combined(X_train, y_dx_train, y_dy_train, X_val, y_dx_val, y_dy_val, input_dim, horizon, config):
#     device = config.DEVICE
#     model = EnhancedSeqModel(input_dim, horizon).to(device)
    
#     # Enable Multi-GPU if available
#     if config.USE_MULTI_GPU:
#         print(f"  Using DataParallel with GPUs: {config.DEVICE_IDS}")
#         model = nn.DataParallel(model, device_ids=config.DEVICE_IDS)
    
    # criterion = EnhancedTemporalLoss(delta=0.5, time_decay=0.05, velocity_weight=0.05)
    # optimizer = torch.optim.AdamW(model.parameters(), lr=config.LEARNING_RATE, weight_decay=1e-4)
    # scheduler = torch.optim.lr_scheduler.OneCycleLR(
    #     optimizer, max_lr=config.LEARNING_RATE, 
    #     epochs=config.EPOCHS, steps_per_epoch=len(X_train)//config.BATCH_SIZE+1
    # )
    
    # train_batches = []
    # for i in range(0, len(X_train), config.BATCH_SIZE):
    #     end = min(i + config.BATCH_SIZE, len(X_train))
    #     bx = torch.tensor(np.stack(X_train[i:end]).astype(np.float32))
    #     by_dx, by_dy, bm = prepare_targets_enhanced(
    #         [y_dx_train[j] for j in range(i, end)],
    #         [y_dy_train[j] for j in range(i, end)], 
    #         horizon
    #     )
    #     train_batches.append((bx, by_dx, by_dy, bm))
    
    # val_batches = []
    # for i in range(0, len(X_val), config.BATCH_SIZE):
    #     end = min(i + config.BATCH_SIZE, len(X_val))
    #     bx = torch.tensor(np.stack(X_val[i:end]).astype(np.float32))
    #     by_dx, by_dy, bm = prepare_targets_enhanced(
    #         [y_dx_val[j] for j in range(i, end)],
    #         [y_dy_val[j] for j in range(i, end)],
    #         horizon
    #     )
    #     val_batches.append((bx, by_dx, by_dy, bm))
    
    # best_rmse, best_state, bad = float('inf'), None, 0
    
    # for epoch in range(1, config.EPOCHS + 1):
    #     model.train()
    #     train_losses = []
        
    #     for bx, by_dx, by_dy, bm in train_batches:
    #         bx, by_dx, by_dy, bm = bx.to(device), by_dx.to(device), by_dy.to(device), bm.to(device)
    #         pred_dx, pred_dy = model(bx)
    #         loss = criterion(pred_dx, pred_dy, by_dx, by_dy, bm)
            
    #         optimizer.zero_grad()
    #         loss.backward()
    #         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    #         optimizer.step()
    #         scheduler.step()
            
    #         train_losses.append(loss.item())
        
    #     model.eval()
    #     val_losses, val_rmses = [], []
    #     with torch.no_grad():
    #         for bx, by_dx, by_dy, bm in val_batches:
    #             bx, by_dx, by_dy, bm = bx.to(device), by_dx.to(device), by_dy.to(device), bm.to(device)
    #             pred_dx, pred_dy = model(bx)
    #             loss = criterion(pred_dx, pred_dy, by_dx, by_dy, bm)
    #             rmse = compute_rmse(pred_dx, pred_dy, by_dx, by_dy, bm)
    #             val_losses.append(loss.item())
    #             val_rmses.append(rmse)
        
    #     train_loss = np.mean(train_losses)
    #     val_loss = np.mean(val_losses)
    #     val_rmse = np.mean(val_rmses)
        
#         if epoch % 10 == 0:
#             lr = scheduler.get_last_lr()[0]
#             print(f"  Epoch {epoch}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, val_rmse={val_rmse:.4f}, lr={lr:.2e}")
        
#         if val_rmse < best_rmse:
#             best_rmse = val_rmse
#             # Handle DataParallel state dict
#             if config.USE_MULTI_GPU:
#                 best_state = {k: v.cpu().clone() for k, v in model.module.state_dict().items()}
#             else:
#                 best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
#             bad = 0
#         else:
#             bad += 1
#             if bad >= config.PATIENCE:
#                 print(f"  Early stop at epoch {epoch}")
#                 break
    
#     if best_state:
#         if config.USE_MULTI_GPU:
#             model.module.load_state_dict(best_state)
#         else:
#             model.load_state_dict(best_state)
    
#     # Return unwrapped model for single-GPU inference
#     return model.module if config.USE_MULTI_GPU else model, best_rmse

# def main():
    # print("\n" + "="*80)
    # print("NFL TRAJECTORY PREDICTION")
    # # print("="*80 + "\n")
    
    # # Load data
    # print("\n[1/4] Loading data...")
    # train_input_files = [Config.DATA_DIR / f"train/input_2023_w{w:02d}.csv" for w in range(1, 19)]
    # train_output_files = [Config.DATA_DIR / f"train/output_2023_w{w:02d}.csv" for w in range(1, 19)]

    # train_input = pd.concat([pd.read_csv(f) for f in train_input_files if f.exists()])
    # train_output = pd.concat([pd.read_csv(f) for f in train_output_files if f.exists()])

    # test_input = pd.read_csv(Config.DATA_DIR / "test_input.csv")
    # test_template = pd.read_csv(Config.DATA_DIR / "test.csv")
    
    # # Prepare features with EXPERT engineering
    # X_seq, y_dx, y_dy, group_ids = prepare_combined_features(
    #     train_input, 
    #     train_output, 
    #     window_size=Config.WINDOW_SIZE,
    #     is_training=True
    # )
    
    # print(f"\n✓ Total sequences: {len(X_seq)}")
    # print(f"✓ Feature dimension: {X_seq[0].shape[1]}")  # Debería ser ~200-250 features
    
    # sequences = np.array(X_seq, dtype=object)
    # targets_dx = np.array(y_dx, dtype=object)
    # targets_dy = np.array(y_dy, dtype=object)
    
    # # Train with combined approach
    # print("\n[3/4] Training COMBINED model...")
    # groups = np.array([d['game_id'] for d in group_ids])
    # gkf = GroupKFold(n_splits=Config.N_FOLDS)
    
    # models, scalers, fold_rmses = [], [], []
    # oof_predictions = np.zeros((len(sequences), Config.MAX_FUTURE_HORIZON, 2))  # Store OOF predictions

    # for fold, (tr, va) in enumerate(gkf.split(sequences, groups=groups), 1):
    #     print(f"\nFold {fold}/{Config.N_FOLDS}")

    #     X_tr = sequences[tr]
    #     X_va = sequences[va]
        
    #     scaler = StandardScaler()
    #     scaler.fit(np.vstack([s for s in X_tr]))
        
    #     X_tr_scaled = np.stack([scaler.transform(s) for s in X_tr])
    #     X_va_scaled = np.stack([scaler.transform(s) for s in X_va])
        
    #     model, val_rmse = train_model_combined(
    #         X_tr_scaled, targets_dx[tr], targets_dy[tr], 
    #         X_va_scaled, targets_dx[va], targets_dy[va],
    #         X_tr[0].shape[-1], Config.MAX_FUTURE_HORIZON, Config
    #     )
        
    #     # Store OOF predictions for validation set
    #     model.eval()
    #     with torch.no_grad():
    #         X_va_tensor = torch.tensor(X_va_scaled.astype(np.float32)).to(Config.DEVICE)
    #         pred_dx, pred_dy = model(X_va_tensor)
    #         oof_predictions[va, :, 0] = pred_dx.cpu().numpy()
    #         oof_predictions[va, :, 1] = pred_dy.cpu().numpy()
        
    #     models.append(model)
    #     scalers.append(scaler)
    #     fold_rmses.append(val_rmse)
        
    #     print(f"Fold {fold} completed with val_RMSE: {val_rmse:.4f}")
    
    # # Calculate overall OOF RMSE
    # print("\n" + "="*80)
    # print("CROSS-VALIDATION RESULTS")
    # print("="*80)
    # for fold, rmse in enumerate(fold_rmses, 1):
    #     print(f"Fold {fold} RMSE: {rmse:.4f}")
    
    # mean_rmse = np.mean(fold_rmses)
    # std_rmse = np.std(fold_rmses)
    # print(f"\nMean CV RMSE: {mean_rmse:.4f} ± {std_rmse:.4f}")
    
    # # Calculate full OOF RMSE (across all folds)
    # all_squared_errors = []
    # for i in range(len(sequences)):
    #     target_dx = targets_dx[i]
    #     target_dy = targets_dy[i]
    #     pred_dx = oof_predictions[i, :len(target_dx), 0]
    #     pred_dy = oof_predictions[i, :len(target_dy), 1]
        
    #     squared_errors = (pred_dx - target_dx)**2 + (pred_dy - target_dy)**2
    #     all_squared_errors.extend(squared_errors)
    
    # oof_rmse = np.sqrt(np.mean(all_squared_errors))
    # print(f"Overall OOF RMSE: {oof_rmse:.4f}")
    # print("="*80 + "\n")
    
    # # Predict
    # print("\n[4/4] Generating final predictions...")
    # test_sequences, test_ids, _ = prepare_combined_features(
    #     test_input, test_template=test_template, is_training=False, window_size=Config.WINDOW_SIZE
    # )
    
    # X_test = np.array(test_sequences, dtype=object)
    # x_last = np.array([s[-1, 0] for s in X_test])
    # y_last = np.array([s[-1, 1] for s in X_test])
    
    # # Ensemble predictions
    # all_dx, all_dy = [], []
    
    # for model, sc in zip(models, scalers):
    #     X_scaled = np.stack([sc.transform(s) for s in X_test])
    #     X_tensor = torch.tensor(X_scaled.astype(np.float32)).to(Config.DEVICE)

    #     model.eval()
    #     with torch.no_grad():
    #         dx, dy = model(X_tensor)
    #         all_dx.append(dx.cpu().numpy())
    #         all_dy.append(dy.cpu().numpy())
    
    # ens_dx = np.mean(all_dx, axis=0)
    # ens_dy = np.mean(all_dy, axis=0)
    
    # # Create submission
    # rows = []
    # H = ens_dx.shape[1]
    
    # for i, sid in enumerate(test_ids):
    #     fids = test_template[
#             (test_template['game_id'] == sid['game_id']) &
#             (test_template['play_id'] == sid['play_id']) &
#             (test_template['nfl_id'] == sid['nfl_id'])
#         ]['frame_id'].sort_values().tolist()
        
#         for t, fid in enumerate(fids):
#             tt = min(t, H - 1)
#             px = np.clip(x_last[i] + ens_dx[i, tt], Config.FIELD_X_MIN, Config.FIELD_X_MAX)
#             py = np.clip(y_last[i] + ens_dy[i, tt], Config.FIELD_Y_MIN, Config.FIELD_Y_MAX)
            
#             rows.append({
#                 'id': f"{sid['game_id']}_{sid['play_id']}_{sid['nfl_id']}_{fid}",
#                 'x': float(px),
#                 'y': float(py)
#             })
    
#     submission = pd.DataFrame(rows)
#     submission.to_csv("submission.csv", index=False)
#     print("="*47)
#     print(f"\n✓ COMBINED submission saved")
#     print(f"  Rows: {len(submission)}")
#     print(f"  Features used: {len(feature_cols)}")
#     print(f"  OOF RMSE: {oof_rmse:.4f}")
#     print(f"  Expected LB RMSE: {oof_rmse:.4f} (±0.01)")
#     print(f"\nKey advantages:")
#     print(f"  • 90+ engineered features for rich representation")
#     print(f"  • Multi-scale model (GRU + Conv1D + Attention)")
#     print(f"  • Velocity-consistent loss for smooth trajectories")
#     print(f"  • OneCycle LR for faster convergence")
#     print(f"  • OOF RMSE tracking for reliable validation")
#     print("="*47)
    
#     return submission

# if __name__ == "__main__":
#     main()